In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import tensorflow as tf
from tensorflow import keras
import import_ipynb
import time
import warnings
warnings.filterwarnings('ignore')
import Lib.Functions as fc

importing Jupyter notebook from /Users/vuhan/Desktop/JOHN/6. Deep Learning with Python/6. Self Book Practice/Natural Language Processing/Lib/Functions.ipynb


In [2]:
data = pd.read_csv('train.csv')
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
data.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [4]:
data['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [7]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [8]:
dup_val = data.shape[0] - data.drop_duplicates().shape[0]

if dup_val > 0:
    print('No duplicate values.')
else:
    print('Duplicate row: {}'.format(dup_val))

Duplicate row: 0


In [12]:
sentences = []
for row in data['text']:
    for word in stopwords:
        token = ' ' + word + ' '
        row = row.replace(token, ' ')
    sentences.append(row)

In [10]:
data['clean_text'] = data['text'].map(lambda x: x.lower())
data.head()

,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this #earthquake m...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask. canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to 'shelter in place' are ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby #alaska as ...


In [11]:
data['length'] = data['text'].map(lambda x: len(x))

In [12]:
data.head()

,id,keyword,location,text,target,clean_text,length
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this #earthquake m...,69
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask. canada,38
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to 'shelter in place' are ...,133
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or...",65
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby #alaska as ...,88


In [13]:
data.groupby('target')['length'].mean()

target
0     95.706817
1    108.113421
Name: length, dtype: float64

In [17]:
X = data.iloc[:, 5]
y = data.iloc[:, 4]

In [18]:
from sklearn.model_selection import train_test_split, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words = 'english')
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [46]:
# from lazypredict.Supervised import LazyClassifier

# clf = LazyClassifier(verbose = 0, ignore_warnings = True, custom_metric = None)
# models, predictions = clf.fit(X_train_cv.toarray(), X_test_cv.toarray(), y_train, y_test)

In [28]:
vocab_size = 1500
max_length = 150
embedding_dim = 128
oov_tok = '<OOV>'
padding_type = 'post'
trunc_type = 'post'

In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

train_sequence = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequence, maxlen = max_length, padding = padding_type, truncating = trunc_type)

valid_sequence = tokenizer.texts_to_sequences(X_test)
valid_padded = pad_sequences(valid_sequence, maxlen = max_length, padding = padding_type, truncating = trunc_type)

In [30]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

history = model.fit(train_padded, y_train,
                    validation_data = (valid_padded, y_test),
                    epochs = 150,
                    verbose = 2)

Epoch 1/150
191/191 - 1s - loss: 0.6796 - accuracy: 0.5662 - val_loss: 0.6565 - val_accuracy: 0.5739 - 931ms/epoch - 5ms/step
Epoch 2/150
191/191 - 0s - loss: 0.5399 - accuracy: 0.7328 - val_loss: 0.5005 - val_accuracy: 0.7794 - 429ms/epoch - 2ms/step
Epoch 3/150
191/191 - 0s - loss: 0.4256 - accuracy: 0.8135 - val_loss: 0.4676 - val_accuracy: 0.7919 - 395ms/epoch - 2ms/step
Epoch 4/150
191/191 - 0s - loss: 0.3841 - accuracy: 0.8356 - val_loss: 0.5352 - val_accuracy: 0.7406 - 389ms/epoch - 2ms/step
Epoch 5/150
191/191 - 0s - loss: 0.3605 - accuracy: 0.8456 - val_loss: 0.4743 - val_accuracy: 0.7951 - 397ms/epoch - 2ms/step
Epoch 6/150
191/191 - 0s - loss: 0.3510 - accuracy: 0.8530 - val_loss: 0.5132 - val_accuracy: 0.7656 - 398ms/epoch - 2ms/step
Epoch 7/150
191/191 - 0s - loss: 0.3424 - accuracy: 0.8558 - val_loss: 0.4967 - val_accuracy: 0.7886 - 393ms/epoch - 2ms/step
Epoch 8/150
191/191 - 0s - loss: 0.3384 - accuracy: 0.8583 - val_loss: 0.4756 - val_accuracy: 0.7886 - 392ms/epoch - 2

Epoch 66/150
191/191 - 0s - loss: 0.1017 - accuracy: 0.9585 - val_loss: 1.7944 - val_accuracy: 0.7452 - 476ms/epoch - 2ms/step
Epoch 67/150
191/191 - 0s - loss: 0.1074 - accuracy: 0.9571 - val_loss: 1.8619 - val_accuracy: 0.7551 - 449ms/epoch - 2ms/step
Epoch 68/150
191/191 - 0s - loss: 0.1035 - accuracy: 0.9576 - val_loss: 1.7943 - val_accuracy: 0.7406 - 464ms/epoch - 2ms/step
Epoch 69/150
191/191 - 0s - loss: 0.1011 - accuracy: 0.9575 - val_loss: 1.8550 - val_accuracy: 0.7255 - 447ms/epoch - 2ms/step
Epoch 70/150
191/191 - 0s - loss: 0.1073 - accuracy: 0.9537 - val_loss: 1.8611 - val_accuracy: 0.7374 - 436ms/epoch - 2ms/step
Epoch 71/150
191/191 - 0s - loss: 0.1074 - accuracy: 0.9537 - val_loss: 1.7480 - val_accuracy: 0.7163 - 464ms/epoch - 2ms/step
Epoch 72/150
191/191 - 0s - loss: 0.0916 - accuracy: 0.9632 - val_loss: 1.8976 - val_accuracy: 0.7341 - 429ms/epoch - 2ms/step
Epoch 73/150
191/191 - 0s - loss: 0.0886 - accuracy: 0.9645 - val_loss: 2.2869 - val_accuracy: 0.7577 - 419ms/e

Epoch 131/150
191/191 - 0s - loss: 0.0526 - accuracy: 0.9754 - val_loss: 2.9164 - val_accuracy: 0.7439 - 438ms/epoch - 2ms/step
Epoch 132/150
191/191 - 0s - loss: 0.0585 - accuracy: 0.9750 - val_loss: 2.8627 - val_accuracy: 0.7380 - 454ms/epoch - 2ms/step
Epoch 133/150
191/191 - 0s - loss: 0.0545 - accuracy: 0.9772 - val_loss: 2.8686 - val_accuracy: 0.7380 - 400ms/epoch - 2ms/step
Epoch 134/150
191/191 - 0s - loss: 0.0491 - accuracy: 0.9788 - val_loss: 3.1395 - val_accuracy: 0.7479 - 419ms/epoch - 2ms/step
Epoch 135/150
191/191 - 0s - loss: 0.0507 - accuracy: 0.9770 - val_loss: 3.0722 - val_accuracy: 0.7393 - 431ms/epoch - 2ms/step
Epoch 136/150
191/191 - 0s - loss: 0.0478 - accuracy: 0.9780 - val_loss: 2.9513 - val_accuracy: 0.7466 - 402ms/epoch - 2ms/step
Epoch 137/150
191/191 - 0s - loss: 0.0484 - accuracy: 0.9782 - val_loss: 3.1631 - val_accuracy: 0.7367 - 460ms/epoch - 2ms/step
Epoch 138/150
191/191 - 1s - loss: 0.0452 - accuracy: 0.9808 - val_loss: 3.1992 - val_accuracy: 0.7413 -

In [31]:
from sklearn.metrics import classification_report

yhat = model.predict(valid_padded)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.75      0.82      0.78       874
           1       0.72      0.64      0.68       649

    accuracy                           0.74      1523
   macro avg       0.74      0.73      0.73      1523
weighted avg       0.74      0.74      0.74      1523



In [32]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Conv1D(32, 3, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(pool_size = 2, strides = 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

history = model.fit(train_padded, y_train,
                    validation_data = (valid_padded, y_test),
                    epochs = 150,
                    verbose = 2)

Epoch 1/150
191/191 - 3s - loss: 0.5814 - accuracy: 0.6875 - val_loss: 0.4640 - val_accuracy: 0.7912 - 3s/epoch - 17ms/step
Epoch 2/150
191/191 - 3s - loss: 0.3952 - accuracy: 0.8268 - val_loss: 0.4686 - val_accuracy: 0.7919 - 3s/epoch - 14ms/step
Epoch 3/150
191/191 - 3s - loss: 0.3126 - accuracy: 0.8711 - val_loss: 0.5233 - val_accuracy: 0.7814 - 3s/epoch - 13ms/step
Epoch 4/150
191/191 - 3s - loss: 0.2292 - accuracy: 0.9174 - val_loss: 0.5802 - val_accuracy: 0.7800 - 3s/epoch - 14ms/step
Epoch 5/150
191/191 - 3s - loss: 0.1621 - accuracy: 0.9476 - val_loss: 0.6697 - val_accuracy: 0.7669 - 3s/epoch - 15ms/step
Epoch 6/150
191/191 - 3s - loss: 0.1200 - accuracy: 0.9637 - val_loss: 0.6907 - val_accuracy: 0.7617 - 3s/epoch - 14ms/step
Epoch 7/150
191/191 - 3s - loss: 0.0961 - accuracy: 0.9709 - val_loss: 0.8057 - val_accuracy: 0.7630 - 3s/epoch - 14ms/step
Epoch 8/150
191/191 - 3s - loss: 0.0842 - accuracy: 0.9739 - val_loss: 0.8622 - val_accuracy: 0.7603 - 3s/epoch - 14ms/step
Epoch 9/

Epoch 67/150
191/191 - 3s - loss: 0.0262 - accuracy: 0.9865 - val_loss: 2.5865 - val_accuracy: 0.7610 - 3s/epoch - 14ms/step
Epoch 68/150
191/191 - 3s - loss: 0.0311 - accuracy: 0.9849 - val_loss: 2.5391 - val_accuracy: 0.7518 - 3s/epoch - 14ms/step
Epoch 69/150
191/191 - 3s - loss: 0.0277 - accuracy: 0.9844 - val_loss: 2.6604 - val_accuracy: 0.7538 - 3s/epoch - 14ms/step
Epoch 70/150
191/191 - 3s - loss: 0.0311 - accuracy: 0.9849 - val_loss: 2.4608 - val_accuracy: 0.7511 - 3s/epoch - 14ms/step
Epoch 71/150
191/191 - 3s - loss: 0.0287 - accuracy: 0.9860 - val_loss: 2.3549 - val_accuracy: 0.7571 - 3s/epoch - 14ms/step
Epoch 72/150
191/191 - 3s - loss: 0.0268 - accuracy: 0.9856 - val_loss: 2.5070 - val_accuracy: 0.7341 - 3s/epoch - 14ms/step
Epoch 73/150
191/191 - 3s - loss: 0.0345 - accuracy: 0.9852 - val_loss: 2.2740 - val_accuracy: 0.7380 - 3s/epoch - 15ms/step
Epoch 74/150
191/191 - 3s - loss: 0.0289 - accuracy: 0.9859 - val_loss: 2.5756 - val_accuracy: 0.7400 - 3s/epoch - 15ms/step


Epoch 133/150
191/191 - 3s - loss: 0.0246 - accuracy: 0.9860 - val_loss: 2.9138 - val_accuracy: 0.7354 - 3s/epoch - 14ms/step
Epoch 134/150
191/191 - 3s - loss: 0.0246 - accuracy: 0.9867 - val_loss: 2.7412 - val_accuracy: 0.7393 - 3s/epoch - 14ms/step
Epoch 135/150
191/191 - 3s - loss: 0.0276 - accuracy: 0.9870 - val_loss: 2.5425 - val_accuracy: 0.7354 - 3s/epoch - 14ms/step
Epoch 136/150
191/191 - 3s - loss: 0.0238 - accuracy: 0.9875 - val_loss: 2.8480 - val_accuracy: 0.7472 - 3s/epoch - 14ms/step
Epoch 137/150
191/191 - 3s - loss: 0.0242 - accuracy: 0.9878 - val_loss: 2.7223 - val_accuracy: 0.7452 - 3s/epoch - 14ms/step
Epoch 138/150
191/191 - 3s - loss: 0.0240 - accuracy: 0.9874 - val_loss: 2.7990 - val_accuracy: 0.7334 - 3s/epoch - 14ms/step
Epoch 139/150
191/191 - 3s - loss: 0.0275 - accuracy: 0.9874 - val_loss: 2.8632 - val_accuracy: 0.7452 - 3s/epoch - 14ms/step
Epoch 140/150
191/191 - 3s - loss: 0.0280 - accuracy: 0.9860 - val_loss: 2.5219 - val_accuracy: 0.7433 - 3s/epoch - 15

In [33]:
yhat = model.predict(valid_padded)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.78      0.79      0.78       874
           1       0.71      0.69      0.70       649

    accuracy                           0.75      1523
   macro avg       0.74      0.74      0.74      1523
weighted avg       0.75      0.75      0.75      1523



In [34]:
tf.keras.backend.clear_session()
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Conv1D(64, 5, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(pool_size = 2, strides = 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation = 'elu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Dense(16, activation = 'elu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

history = model.fit(train_padded, y_train,
                    validation_data = (valid_padded, y_test),
                    epochs = 150,
                    verbose = 2)

Epoch 1/150
191/191 - 4s - loss: 0.5450 - accuracy: 0.7271 - val_loss: 0.4558 - val_accuracy: 0.7984 - 4s/epoch - 21ms/step
Epoch 2/150
191/191 - 4s - loss: 0.3654 - accuracy: 0.8440 - val_loss: 0.4907 - val_accuracy: 0.7912 - 4s/epoch - 18ms/step
Epoch 3/150
191/191 - 4s - loss: 0.2540 - accuracy: 0.9018 - val_loss: 0.5971 - val_accuracy: 0.7833 - 4s/epoch - 19ms/step
Epoch 4/150
191/191 - 4s - loss: 0.1512 - accuracy: 0.9521 - val_loss: 0.7497 - val_accuracy: 0.7518 - 4s/epoch - 18ms/step
Epoch 5/150
191/191 - 3s - loss: 0.1088 - accuracy: 0.9681 - val_loss: 0.8185 - val_accuracy: 0.7479 - 3s/epoch - 18ms/step
Epoch 6/150
191/191 - 4s - loss: 0.0814 - accuracy: 0.9729 - val_loss: 0.8119 - val_accuracy: 0.7623 - 4s/epoch - 18ms/step
Epoch 7/150
191/191 - 4s - loss: 0.0682 - accuracy: 0.9752 - val_loss: 0.9731 - val_accuracy: 0.7630 - 4s/epoch - 18ms/step
Epoch 8/150
191/191 - 4s - loss: 0.0532 - accuracy: 0.9780 - val_loss: 1.0618 - val_accuracy: 0.7617 - 4s/epoch - 18ms/step
Epoch 9/

Epoch 67/150
191/191 - 5s - loss: 0.0249 - accuracy: 0.9860 - val_loss: 2.3281 - val_accuracy: 0.7466 - 5s/epoch - 24ms/step
Epoch 68/150
191/191 - 5s - loss: 0.0243 - accuracy: 0.9870 - val_loss: 2.6392 - val_accuracy: 0.7511 - 5s/epoch - 26ms/step
Epoch 69/150
191/191 - 6s - loss: 0.0250 - accuracy: 0.9867 - val_loss: 2.5109 - val_accuracy: 0.7380 - 6s/epoch - 30ms/step
Epoch 70/150
191/191 - 6s - loss: 0.0306 - accuracy: 0.9846 - val_loss: 2.0922 - val_accuracy: 0.7413 - 6s/epoch - 31ms/step
Epoch 71/150
191/191 - 6s - loss: 0.0382 - accuracy: 0.9831 - val_loss: 1.7400 - val_accuracy: 0.7571 - 6s/epoch - 30ms/step
Epoch 72/150
191/191 - 6s - loss: 0.0271 - accuracy: 0.9847 - val_loss: 2.0328 - val_accuracy: 0.7551 - 6s/epoch - 29ms/step
Epoch 73/150
191/191 - 6s - loss: 0.0255 - accuracy: 0.9854 - val_loss: 2.1896 - val_accuracy: 0.7551 - 6s/epoch - 30ms/step
Epoch 74/150
191/191 - 6s - loss: 0.0294 - accuracy: 0.9847 - val_loss: 2.2353 - val_accuracy: 0.7518 - 6s/epoch - 29ms/step


Epoch 133/150
191/191 - 6s - loss: 0.0226 - accuracy: 0.9877 - val_loss: 2.4100 - val_accuracy: 0.7538 - 6s/epoch - 31ms/step
Epoch 134/150
191/191 - 6s - loss: 0.0223 - accuracy: 0.9872 - val_loss: 2.5267 - val_accuracy: 0.7518 - 6s/epoch - 31ms/step
Epoch 135/150
191/191 - 6s - loss: 0.0225 - accuracy: 0.9872 - val_loss: 2.5182 - val_accuracy: 0.7518 - 6s/epoch - 31ms/step
Epoch 136/150
191/191 - 6s - loss: 0.0226 - accuracy: 0.9874 - val_loss: 2.5855 - val_accuracy: 0.7518 - 6s/epoch - 29ms/step
Epoch 137/150
191/191 - 5s - loss: 0.0224 - accuracy: 0.9869 - val_loss: 2.9502 - val_accuracy: 0.7505 - 5s/epoch - 28ms/step
Epoch 138/150
191/191 - 5s - loss: 0.0233 - accuracy: 0.9869 - val_loss: 3.1026 - val_accuracy: 0.7433 - 5s/epoch - 28ms/step
Epoch 139/150
191/191 - 5s - loss: 0.0323 - accuracy: 0.9857 - val_loss: 2.2450 - val_accuracy: 0.7584 - 5s/epoch - 28ms/step
Epoch 140/150
191/191 - 5s - loss: 0.0263 - accuracy: 0.9857 - val_loss: 2.2051 - val_accuracy: 0.7617 - 5s/epoch - 28

In [35]:
yhat = model.predict(valid_padded)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.77      0.82      0.80       874
           1       0.74      0.67      0.70       649

    accuracy                           0.76      1523
   macro avg       0.76      0.75      0.75      1523
weighted avg       0.76      0.76      0.76      1523



In [115]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Conv1D(32, 3, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(pool_size = 2, strides = 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(24, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

history = model.fit(train_padded, y_train,
                    validation_data = (valid_padded, y_test),
                    epochs = 150,
                    verbose = 2)

Epoch 1/150
191/191 - 2s - loss: 0.6388 - accuracy: 0.6278 - val_loss: 0.4994 - val_accuracy: 0.7741 - 2s/epoch - 11ms/step
Epoch 2/150
191/191 - 2s - loss: 0.4368 - accuracy: 0.8021 - val_loss: 0.4720 - val_accuracy: 0.7945 - 2s/epoch - 8ms/step
Epoch 3/150
191/191 - 2s - loss: 0.3749 - accuracy: 0.8371 - val_loss: 0.5228 - val_accuracy: 0.7768 - 2s/epoch - 8ms/step
Epoch 4/150
191/191 - 2s - loss: 0.3439 - accuracy: 0.8548 - val_loss: 0.4904 - val_accuracy: 0.7925 - 2s/epoch - 8ms/step
Epoch 5/150
191/191 - 2s - loss: 0.3068 - accuracy: 0.8749 - val_loss: 0.5129 - val_accuracy: 0.7892 - 2s/epoch - 8ms/step
Epoch 6/150
191/191 - 2s - loss: 0.2619 - accuracy: 0.8975 - val_loss: 0.5360 - val_accuracy: 0.7728 - 2s/epoch - 8ms/step
Epoch 7/150
191/191 - 2s - loss: 0.2335 - accuracy: 0.9112 - val_loss: 0.5772 - val_accuracy: 0.7774 - 2s/epoch - 9ms/step
Epoch 8/150
191/191 - 2s - loss: 0.1916 - accuracy: 0.9274 - val_loss: 0.6287 - val_accuracy: 0.7748 - 2s/epoch - 8ms/step
Epoch 9/150
191

Epoch 68/150
191/191 - 2s - loss: 0.0422 - accuracy: 0.9813 - val_loss: 2.2333 - val_accuracy: 0.7623 - 2s/epoch - 8ms/step
Epoch 69/150
191/191 - 2s - loss: 0.0434 - accuracy: 0.9811 - val_loss: 2.2663 - val_accuracy: 0.7649 - 2s/epoch - 8ms/step
Epoch 70/150
191/191 - 2s - loss: 0.0456 - accuracy: 0.9793 - val_loss: 2.0452 - val_accuracy: 0.7571 - 2s/epoch - 8ms/step
Epoch 71/150
191/191 - 2s - loss: 0.0414 - accuracy: 0.9828 - val_loss: 2.1690 - val_accuracy: 0.7571 - 2s/epoch - 8ms/step
Epoch 72/150
191/191 - 2s - loss: 0.0456 - accuracy: 0.9795 - val_loss: 2.0239 - val_accuracy: 0.7643 - 2s/epoch - 8ms/step
Epoch 73/150
191/191 - 2s - loss: 0.0401 - accuracy: 0.9806 - val_loss: 2.2270 - val_accuracy: 0.7610 - 2s/epoch - 8ms/step
Epoch 74/150
191/191 - 2s - loss: 0.0420 - accuracy: 0.9806 - val_loss: 2.2198 - val_accuracy: 0.7636 - 2s/epoch - 9ms/step
Epoch 75/150
191/191 - 2s - loss: 0.0386 - accuracy: 0.9818 - val_loss: 2.2304 - val_accuracy: 0.7630 - 2s/epoch - 8ms/step
Epoch 76

Epoch 134/150
191/191 - 2s - loss: 0.0377 - accuracy: 0.9829 - val_loss: 2.5747 - val_accuracy: 0.7439 - 2s/epoch - 9ms/step
Epoch 135/150
191/191 - 2s - loss: 0.0346 - accuracy: 0.9831 - val_loss: 2.6558 - val_accuracy: 0.7590 - 2s/epoch - 9ms/step
Epoch 136/150
191/191 - 2s - loss: 0.0387 - accuracy: 0.9823 - val_loss: 2.5627 - val_accuracy: 0.7505 - 2s/epoch - 9ms/step
Epoch 137/150
191/191 - 2s - loss: 0.0324 - accuracy: 0.9846 - val_loss: 2.6773 - val_accuracy: 0.7531 - 2s/epoch - 9ms/step
Epoch 138/150
191/191 - 2s - loss: 0.0374 - accuracy: 0.9826 - val_loss: 2.4936 - val_accuracy: 0.7459 - 2s/epoch - 9ms/step
Epoch 139/150
191/191 - 2s - loss: 0.0350 - accuracy: 0.9810 - val_loss: 2.6698 - val_accuracy: 0.7459 - 2s/epoch - 9ms/step
Epoch 140/150
191/191 - 2s - loss: 0.0330 - accuracy: 0.9847 - val_loss: 2.8601 - val_accuracy: 0.7400 - 2s/epoch - 10ms/step
Epoch 141/150
191/191 - 2s - loss: 0.0341 - accuracy: 0.9828 - val_loss: 2.7943 - val_accuracy: 0.7452 - 2s/epoch - 10ms/ste

In [116]:
yhat = model.predict(valid_padded)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.79      0.78      0.78       874
           1       0.71      0.72      0.71       649

    accuracy                           0.75      1523
   macro avg       0.75      0.75      0.75      1523
weighted avg       0.75      0.75      0.75      1523



In [117]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

history = model.fit(train_padded, y_train,
                    validation_data = (valid_padded, y_test),
                    epochs = 150,
                    verbose = 2)

Epoch 1/150
191/191 - 1s - loss: 0.6812 - accuracy: 0.5658 - val_loss: 0.6743 - val_accuracy: 0.5739 - 771ms/epoch - 4ms/step
Epoch 2/150
191/191 - 0s - loss: 0.6535 - accuracy: 0.5967 - val_loss: 0.6231 - val_accuracy: 0.7295 - 242ms/epoch - 1ms/step
Epoch 3/150
191/191 - 0s - loss: 0.5643 - accuracy: 0.7274 - val_loss: 0.5607 - val_accuracy: 0.7242 - 250ms/epoch - 1ms/step
Epoch 4/150
191/191 - 0s - loss: 0.4873 - accuracy: 0.7772 - val_loss: 0.4973 - val_accuracy: 0.7853 - 253ms/epoch - 1ms/step
Epoch 5/150
191/191 - 0s - loss: 0.4413 - accuracy: 0.8085 - val_loss: 0.4807 - val_accuracy: 0.7912 - 256ms/epoch - 1ms/step
Epoch 6/150
191/191 - 0s - loss: 0.4151 - accuracy: 0.8220 - val_loss: 0.4763 - val_accuracy: 0.7945 - 254ms/epoch - 1ms/step
Epoch 7/150
191/191 - 0s - loss: 0.3982 - accuracy: 0.8302 - val_loss: 0.4713 - val_accuracy: 0.7978 - 307ms/epoch - 2ms/step
Epoch 8/150
191/191 - 0s - loss: 0.3839 - accuracy: 0.8363 - val_loss: 0.4875 - val_accuracy: 0.7787 - 308ms/epoch - 2

Epoch 66/150
191/191 - 0s - loss: 0.2958 - accuracy: 0.8727 - val_loss: 0.5971 - val_accuracy: 0.7761 - 322ms/epoch - 2ms/step
Epoch 67/150
191/191 - 0s - loss: 0.2977 - accuracy: 0.8785 - val_loss: 0.6074 - val_accuracy: 0.7735 - 300ms/epoch - 2ms/step
Epoch 68/150
191/191 - 0s - loss: 0.2999 - accuracy: 0.8726 - val_loss: 0.6013 - val_accuracy: 0.7761 - 321ms/epoch - 2ms/step
Epoch 69/150
191/191 - 0s - loss: 0.2991 - accuracy: 0.8718 - val_loss: 0.6148 - val_accuracy: 0.7748 - 259ms/epoch - 1ms/step
Epoch 70/150
191/191 - 0s - loss: 0.2975 - accuracy: 0.8714 - val_loss: 0.6264 - val_accuracy: 0.7695 - 294ms/epoch - 2ms/step
Epoch 71/150
191/191 - 0s - loss: 0.2924 - accuracy: 0.8783 - val_loss: 0.6122 - val_accuracy: 0.7754 - 261ms/epoch - 1ms/step
Epoch 72/150
191/191 - 0s - loss: 0.2927 - accuracy: 0.8795 - val_loss: 0.6087 - val_accuracy: 0.7708 - 270ms/epoch - 1ms/step
Epoch 73/150
191/191 - 0s - loss: 0.2942 - accuracy: 0.8788 - val_loss: 0.6332 - val_accuracy: 0.7689 - 322ms/e

Epoch 131/150
191/191 - 0s - loss: 0.2195 - accuracy: 0.9087 - val_loss: 0.8193 - val_accuracy: 0.7708 - 257ms/epoch - 1ms/step
Epoch 132/150
191/191 - 0s - loss: 0.2185 - accuracy: 0.9087 - val_loss: 0.8757 - val_accuracy: 0.7669 - 262ms/epoch - 1ms/step
Epoch 133/150
191/191 - 0s - loss: 0.2106 - accuracy: 0.9130 - val_loss: 0.8481 - val_accuracy: 0.7702 - 257ms/epoch - 1ms/step
Epoch 134/150
191/191 - 0s - loss: 0.2132 - accuracy: 0.9123 - val_loss: 0.9047 - val_accuracy: 0.7663 - 276ms/epoch - 1ms/step
Epoch 135/150
191/191 - 0s - loss: 0.2085 - accuracy: 0.9149 - val_loss: 0.9310 - val_accuracy: 0.7623 - 264ms/epoch - 1ms/step
Epoch 136/150
191/191 - 0s - loss: 0.2080 - accuracy: 0.9130 - val_loss: 0.9245 - val_accuracy: 0.7643 - 257ms/epoch - 1ms/step
Epoch 137/150
191/191 - 0s - loss: 0.2045 - accuracy: 0.9158 - val_loss: 0.8975 - val_accuracy: 0.7669 - 278ms/epoch - 1ms/step
Epoch 138/150
191/191 - 0s - loss: 0.2054 - accuracy: 0.9172 - val_loss: 0.9853 - val_accuracy: 0.7571 -

In [118]:
yhat = model.predict(valid_padded)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.77      0.86      0.81       874
           1       0.77      0.65      0.71       649

    accuracy                           0.77      1523
   macro avg       0.77      0.75      0.76      1523
weighted avg       0.77      0.77      0.77      1523



In [119]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

history = model.fit(train_padded, y_train,
                    validation_data = (valid_padded, y_test),
                    epochs = 150,
                    verbose = 2)

Epoch 1/150
191/191 - 1s - loss: 0.6799 - accuracy: 0.5655 - val_loss: 0.6717 - val_accuracy: 0.5739 - 793ms/epoch - 4ms/step
Epoch 2/150
191/191 - 0s - loss: 0.6573 - accuracy: 0.5785 - val_loss: 0.6388 - val_accuracy: 0.6218 - 222ms/epoch - 1ms/step
Epoch 3/150
191/191 - 0s - loss: 0.6166 - accuracy: 0.6765 - val_loss: 0.6032 - val_accuracy: 0.6855 - 278ms/epoch - 1ms/step
Epoch 4/150
191/191 - 0s - loss: 0.5756 - accuracy: 0.7343 - val_loss: 0.5719 - val_accuracy: 0.7452 - 248ms/epoch - 1ms/step
Epoch 5/150
191/191 - 0s - loss: 0.5303 - accuracy: 0.7629 - val_loss: 0.5330 - val_accuracy: 0.7735 - 241ms/epoch - 1ms/step
Epoch 6/150
191/191 - 0s - loss: 0.4914 - accuracy: 0.7865 - val_loss: 0.5289 - val_accuracy: 0.7492 - 232ms/epoch - 1ms/step
Epoch 7/150
191/191 - 0s - loss: 0.4624 - accuracy: 0.8031 - val_loss: 0.4976 - val_accuracy: 0.7820 - 233ms/epoch - 1ms/step
Epoch 8/150
191/191 - 0s - loss: 0.4426 - accuracy: 0.8118 - val_loss: 0.5173 - val_accuracy: 0.7439 - 235ms/epoch - 1

Epoch 66/150
191/191 - 0s - loss: 0.3514 - accuracy: 0.8502 - val_loss: 0.5011 - val_accuracy: 0.7873 - 294ms/epoch - 2ms/step
Epoch 67/150
191/191 - 0s - loss: 0.3496 - accuracy: 0.8499 - val_loss: 0.5015 - val_accuracy: 0.7892 - 259ms/epoch - 1ms/step
Epoch 68/150
191/191 - 0s - loss: 0.3500 - accuracy: 0.8537 - val_loss: 0.5097 - val_accuracy: 0.7905 - 285ms/epoch - 1ms/step
Epoch 69/150
191/191 - 0s - loss: 0.3497 - accuracy: 0.8496 - val_loss: 0.5021 - val_accuracy: 0.7912 - 243ms/epoch - 1ms/step
Epoch 70/150
191/191 - 0s - loss: 0.3494 - accuracy: 0.8552 - val_loss: 0.5042 - val_accuracy: 0.7912 - 240ms/epoch - 1ms/step
Epoch 71/150
191/191 - 0s - loss: 0.3490 - accuracy: 0.8539 - val_loss: 0.5048 - val_accuracy: 0.7866 - 238ms/epoch - 1ms/step
Epoch 72/150
191/191 - 0s - loss: 0.3494 - accuracy: 0.8537 - val_loss: 0.5112 - val_accuracy: 0.7702 - 228ms/epoch - 1ms/step
Epoch 73/150
191/191 - 0s - loss: 0.3495 - accuracy: 0.8532 - val_loss: 0.5014 - val_accuracy: 0.7965 - 272ms/e

Epoch 131/150
191/191 - 0s - loss: 0.3306 - accuracy: 0.8686 - val_loss: 0.5174 - val_accuracy: 0.7945 - 241ms/epoch - 1ms/step
Epoch 132/150
191/191 - 0s - loss: 0.3297 - accuracy: 0.8673 - val_loss: 0.5073 - val_accuracy: 0.7912 - 281ms/epoch - 1ms/step
Epoch 133/150
191/191 - 0s - loss: 0.3296 - accuracy: 0.8683 - val_loss: 0.5063 - val_accuracy: 0.7932 - 249ms/epoch - 1ms/step
Epoch 134/150
191/191 - 0s - loss: 0.3293 - accuracy: 0.8686 - val_loss: 0.5140 - val_accuracy: 0.7912 - 228ms/epoch - 1ms/step
Epoch 135/150
191/191 - 0s - loss: 0.3289 - accuracy: 0.8680 - val_loss: 0.5208 - val_accuracy: 0.7925 - 318ms/epoch - 2ms/step
Epoch 136/150
191/191 - 0s - loss: 0.3281 - accuracy: 0.8691 - val_loss: 0.5007 - val_accuracy: 0.7945 - 238ms/epoch - 1ms/step
Epoch 137/150
191/191 - 0s - loss: 0.3269 - accuracy: 0.8698 - val_loss: 0.5122 - val_accuracy: 0.7932 - 230ms/epoch - 1ms/step
Epoch 138/150
191/191 - 0s - loss: 0.3282 - accuracy: 0.8703 - val_loss: 0.5342 - val_accuracy: 0.7925 -

In [120]:
yhat = model.predict(valid_padded)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.79      0.89      0.83       874
           1       0.82      0.68      0.74       649

    accuracy                           0.80      1523
   macro avg       0.80      0.78      0.79      1523
weighted avg       0.80      0.80      0.79      1523



In [121]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

history = model.fit(train_padded, y_train,
                    validation_data = (valid_padded, y_test),
                    epochs = 150,
                    verbose = 2)

Epoch 1/150
191/191 - 1s - loss: 0.6806 - accuracy: 0.5672 - val_loss: 0.6706 - val_accuracy: 0.5739 - 788ms/epoch - 4ms/step
Epoch 2/150
191/191 - 0s - loss: 0.6437 - accuracy: 0.6251 - val_loss: 0.6075 - val_accuracy: 0.7203 - 233ms/epoch - 1ms/step
Epoch 3/150
191/191 - 0s - loss: 0.5643 - accuracy: 0.7264 - val_loss: 0.5484 - val_accuracy: 0.7446 - 240ms/epoch - 1ms/step
Epoch 4/150
191/191 - 0s - loss: 0.5069 - accuracy: 0.7626 - val_loss: 0.5317 - val_accuracy: 0.7387 - 240ms/epoch - 1ms/step
Epoch 5/150
191/191 - 0s - loss: 0.4657 - accuracy: 0.7888 - val_loss: 0.4920 - val_accuracy: 0.7807 - 241ms/epoch - 1ms/step
Epoch 6/150
191/191 - 0s - loss: 0.4403 - accuracy: 0.8046 - val_loss: 0.5159 - val_accuracy: 0.7380 - 237ms/epoch - 1ms/step
Epoch 7/150
191/191 - 0s - loss: 0.4226 - accuracy: 0.8141 - val_loss: 0.4900 - val_accuracy: 0.7702 - 235ms/epoch - 1ms/step
Epoch 8/150
191/191 - 0s - loss: 0.4126 - accuracy: 0.8225 - val_loss: 0.5205 - val_accuracy: 0.7406 - 243ms/epoch - 1

Epoch 66/150
191/191 - 0s - loss: 0.3316 - accuracy: 0.8652 - val_loss: 0.5180 - val_accuracy: 0.7971 - 243ms/epoch - 1ms/step
Epoch 67/150
191/191 - 0s - loss: 0.3311 - accuracy: 0.8662 - val_loss: 0.5310 - val_accuracy: 0.7945 - 238ms/epoch - 1ms/step
Epoch 68/150
191/191 - 0s - loss: 0.3301 - accuracy: 0.8657 - val_loss: 0.5170 - val_accuracy: 0.7965 - 265ms/epoch - 1ms/step
Epoch 69/150
191/191 - 0s - loss: 0.3303 - accuracy: 0.8678 - val_loss: 0.5213 - val_accuracy: 0.7971 - 236ms/epoch - 1ms/step
Epoch 70/150
191/191 - 0s - loss: 0.3297 - accuracy: 0.8657 - val_loss: 0.5210 - val_accuracy: 0.7958 - 267ms/epoch - 1ms/step
Epoch 71/150
191/191 - 0s - loss: 0.3293 - accuracy: 0.8672 - val_loss: 0.5373 - val_accuracy: 0.7938 - 243ms/epoch - 1ms/step
Epoch 72/150
191/191 - 0s - loss: 0.3288 - accuracy: 0.8677 - val_loss: 0.5433 - val_accuracy: 0.7846 - 258ms/epoch - 1ms/step
Epoch 73/150
191/191 - 0s - loss: 0.3293 - accuracy: 0.8678 - val_loss: 0.5236 - val_accuracy: 0.7984 - 234ms/e

Epoch 131/150
191/191 - 0s - loss: 0.3036 - accuracy: 0.8810 - val_loss: 0.5844 - val_accuracy: 0.7978 - 261ms/epoch - 1ms/step
Epoch 132/150
191/191 - 0s - loss: 0.3024 - accuracy: 0.8829 - val_loss: 0.5667 - val_accuracy: 0.7879 - 239ms/epoch - 1ms/step
Epoch 133/150
191/191 - 0s - loss: 0.3010 - accuracy: 0.8811 - val_loss: 0.5730 - val_accuracy: 0.7886 - 240ms/epoch - 1ms/step
Epoch 134/150
191/191 - 0s - loss: 0.3011 - accuracy: 0.8805 - val_loss: 0.5692 - val_accuracy: 0.7853 - 259ms/epoch - 1ms/step
Epoch 135/150
191/191 - 0s - loss: 0.3007 - accuracy: 0.8805 - val_loss: 0.5869 - val_accuracy: 0.7919 - 236ms/epoch - 1ms/step
Epoch 136/150
191/191 - 0s - loss: 0.3004 - accuracy: 0.8818 - val_loss: 0.5647 - val_accuracy: 0.7840 - 273ms/epoch - 1ms/step
Epoch 137/150
191/191 - 0s - loss: 0.2991 - accuracy: 0.8819 - val_loss: 0.5724 - val_accuracy: 0.7879 - 240ms/epoch - 1ms/step
Epoch 138/150
191/191 - 0s - loss: 0.2991 - accuracy: 0.8828 - val_loss: 0.6154 - val_accuracy: 0.7892 -

In [122]:
yhat = model.predict(valid_padded)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.79      0.88      0.83       874
           1       0.80      0.68      0.74       649

    accuracy                           0.79      1523
   macro avg       0.80      0.78      0.78      1523
weighted avg       0.79      0.79      0.79      1523



In [125]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

t1 = time.time()
history = model.fit(train_padded, y_train,
                    validation_data = (valid_padded, y_test),
                    epochs = 150,
                    verbose = 2)
t2 = time.time()

print('\nTraining took: {:.2f} minutes'.format((t2 - t1) / 60))

Epoch 1/150
191/191 - 10s - loss: 0.5530 - accuracy: 0.7182 - val_loss: 0.4695 - val_accuracy: 0.7827 - 10s/epoch - 54ms/step
Epoch 2/150
191/191 - 8s - loss: 0.4202 - accuracy: 0.8084 - val_loss: 0.4536 - val_accuracy: 0.8004 - 8s/epoch - 39ms/step
Epoch 3/150
191/191 - 8s - loss: 0.3919 - accuracy: 0.8291 - val_loss: 0.4527 - val_accuracy: 0.8043 - 8s/epoch - 41ms/step
Epoch 4/150
191/191 - 8s - loss: 0.3767 - accuracy: 0.8414 - val_loss: 0.5072 - val_accuracy: 0.7715 - 8s/epoch - 42ms/step
Epoch 5/150
191/191 - 8s - loss: 0.3687 - accuracy: 0.8427 - val_loss: 0.4780 - val_accuracy: 0.7984 - 8s/epoch - 44ms/step
Epoch 6/150
191/191 - 9s - loss: 0.3606 - accuracy: 0.8489 - val_loss: 0.4641 - val_accuracy: 0.7919 - 9s/epoch - 45ms/step
Epoch 7/150
191/191 - 9s - loss: 0.3505 - accuracy: 0.8514 - val_loss: 0.5033 - val_accuracy: 0.7932 - 9s/epoch - 47ms/step
Epoch 8/150
191/191 - 8s - loss: 0.3394 - accuracy: 0.8586 - val_loss: 0.4918 - val_accuracy: 0.7748 - 8s/epoch - 44ms/step
Epoch 

Epoch 67/150
191/191 - 9s - loss: 0.0636 - accuracy: 0.9778 - val_loss: 1.7262 - val_accuracy: 0.7321 - 9s/epoch - 45ms/step
Epoch 68/150
191/191 - 8s - loss: 0.0651 - accuracy: 0.9759 - val_loss: 1.6101 - val_accuracy: 0.7354 - 8s/epoch - 43ms/step
Epoch 69/150
191/191 - 9s - loss: 0.0613 - accuracy: 0.9770 - val_loss: 1.6824 - val_accuracy: 0.7288 - 9s/epoch - 45ms/step
Epoch 70/150
191/191 - 8s - loss: 0.0661 - accuracy: 0.9755 - val_loss: 1.5559 - val_accuracy: 0.7367 - 8s/epoch - 44ms/step
Epoch 71/150
191/191 - 9s - loss: 0.0625 - accuracy: 0.9782 - val_loss: 1.6037 - val_accuracy: 0.7262 - 9s/epoch - 47ms/step
Epoch 72/150
191/191 - 9s - loss: 0.0631 - accuracy: 0.9780 - val_loss: 1.6134 - val_accuracy: 0.7249 - 9s/epoch - 49ms/step
Epoch 73/150
191/191 - 10s - loss: 0.0626 - accuracy: 0.9787 - val_loss: 1.5595 - val_accuracy: 0.7216 - 10s/epoch - 53ms/step
Epoch 74/150
191/191 - 9s - loss: 0.0609 - accuracy: 0.9772 - val_loss: 1.5557 - val_accuracy: 0.7367 - 9s/epoch - 49ms/ste

Epoch 133/150
191/191 - 9s - loss: 0.0455 - accuracy: 0.9824 - val_loss: 1.9131 - val_accuracy: 0.7223 - 9s/epoch - 47ms/step
Epoch 134/150
191/191 - 9s - loss: 0.0458 - accuracy: 0.9819 - val_loss: 1.9525 - val_accuracy: 0.7157 - 9s/epoch - 46ms/step
Epoch 135/150
191/191 - 8s - loss: 0.0442 - accuracy: 0.9813 - val_loss: 1.8845 - val_accuracy: 0.7190 - 8s/epoch - 44ms/step
Epoch 136/150
191/191 - 9s - loss: 0.0469 - accuracy: 0.9813 - val_loss: 1.8690 - val_accuracy: 0.7131 - 9s/epoch - 47ms/step
Epoch 137/150
191/191 - 10s - loss: 0.0459 - accuracy: 0.9818 - val_loss: 2.2192 - val_accuracy: 0.7413 - 10s/epoch - 52ms/step
Epoch 138/150
191/191 - 9s - loss: 0.0459 - accuracy: 0.9828 - val_loss: 1.9464 - val_accuracy: 0.7157 - 9s/epoch - 50ms/step
Epoch 139/150
191/191 - 9s - loss: 0.0468 - accuracy: 0.9821 - val_loss: 1.7581 - val_accuracy: 0.7301 - 9s/epoch - 46ms/step
Epoch 140/150
191/191 - 9s - loss: 0.0452 - accuracy: 0.9816 - val_loss: 2.0641 - val_accuracy: 0.7144 - 9s/epoch - 

In [126]:
yhat = model.predict(valid_padded)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.76      0.77      0.76       874
           1       0.68      0.66      0.67       649

    accuracy                           0.73      1523
   macro avg       0.72      0.72      0.72      1523
weighted avg       0.72      0.73      0.73      1523



# Resample traning set

In [36]:
df_train = pd.DataFrame(X_train)
df_train['target'] = y_train
df_train.head()

,clean_text,target
4996,courageous and honest analysis of need to use ...,1
3263,@zachzaidman @670thescore wld b a shame if tha...,0
4907,tell @barackobama to rescind medals of 'honor'...,1
2855,worried about how the ca drought might affect ...,1
4716,@youngheroesid lava blast &amp; power red #pan...,0


In [37]:
from sklearn.utils import resample

df0_new = df_train[df_train['target'] == 0]
df1_new = df_train[df_train['target'] == 1]

In [38]:
df0_new.shape, df1_new.shape

((3468, 2), (2622, 2))

In [39]:
df1_resample = resample(df1_new, replace=True, 
                        n_samples=df0_new.shape[0], random_state=0)

In [40]:
df1_resample.shape

(3468, 2)

In [41]:
df = pd.concat([df0_new, df1_resample])
df.head()

,clean_text,target
3263,@zachzaidman @670thescore wld b a shame if tha...,0
4716,@youngheroesid lava blast &amp; power red #pan...,0
7173,@robertoneill31 getting hit by a foul ball whi...,0
347,one direction is my pick for http://t.co/q2ebl...,0
333,sadly how windows 10 reveals microsoft's ethic...,0


In [42]:
df['target'].value_counts()

1    3468
0    3468
Name: target, dtype: int64

In [43]:
X1 = df.iloc[:, 0]
y1 = df.iloc[:, -1]

In [44]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.3, random_state = 0)

In [45]:
vocab_size = 1500
max_length = 150
embedding_dim = 128
oov_tok = '<OOV>'
padding_type = 'post'
trunc_type = 'post'

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(X1_train)
word_index = tokenizer.word_index

train_sequence1 = tokenizer.texts_to_sequences(X1_train)
train_padded1 = pad_sequences(train_sequence1, maxlen = max_length, padding = padding_type, truncating = trunc_type)

valid_sequence1 = tokenizer.texts_to_sequences(X1_test)
valid_padded1 = pad_sequences(valid_sequence1, maxlen = max_length, padding = padding_type, truncating = trunc_type)

In [46]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])
t1 = time.time()
history = model.fit(train_padded1, y1_train,
                    validation_data = (valid_padded1, y1_test),
                    epochs = 150,
                    verbose = 2)
t2 = time.time()

print('\nTraining took: {:.2f} minutes'.format((t2 - t1) / 60))

Epoch 1/150
152/152 - 1s - loss: 0.6889 - accuracy: 0.5384 - val_loss: 0.6738 - val_accuracy: 0.7078 - 1s/epoch - 9ms/step
Epoch 2/150
152/152 - 1s - loss: 0.6321 - accuracy: 0.6970 - val_loss: 0.5741 - val_accuracy: 0.7271 - 526ms/epoch - 3ms/step
Epoch 3/150
152/152 - 0s - loss: 0.5403 - accuracy: 0.7413 - val_loss: 0.4989 - val_accuracy: 0.7732 - 498ms/epoch - 3ms/step
Epoch 4/150
152/152 - 0s - loss: 0.4787 - accuracy: 0.7740 - val_loss: 0.4579 - val_accuracy: 0.7982 - 492ms/epoch - 3ms/step
Epoch 5/150
152/152 - 0s - loss: 0.4339 - accuracy: 0.8014 - val_loss: 0.4469 - val_accuracy: 0.8001 - 493ms/epoch - 3ms/step
Epoch 6/150
152/152 - 1s - loss: 0.4030 - accuracy: 0.8198 - val_loss: 0.4309 - val_accuracy: 0.8068 - 546ms/epoch - 4ms/step
Epoch 7/150
152/152 - 1s - loss: 0.3848 - accuracy: 0.8286 - val_loss: 0.4263 - val_accuracy: 0.8087 - 502ms/epoch - 3ms/step
Epoch 8/150
152/152 - 1s - loss: 0.3726 - accuracy: 0.8330 - val_loss: 0.4052 - val_accuracy: 0.8328 - 505ms/epoch - 3ms/

Epoch 66/150
152/152 - 0s - loss: 0.2257 - accuracy: 0.9079 - val_loss: 0.6943 - val_accuracy: 0.7593 - 498ms/epoch - 3ms/step
Epoch 67/150
152/152 - 0s - loss: 0.2277 - accuracy: 0.9069 - val_loss: 0.4508 - val_accuracy: 0.8337 - 466ms/epoch - 3ms/step
Epoch 68/150
152/152 - 0s - loss: 0.2259 - accuracy: 0.9044 - val_loss: 0.4736 - val_accuracy: 0.8289 - 489ms/epoch - 3ms/step
Epoch 69/150
152/152 - 1s - loss: 0.2229 - accuracy: 0.9096 - val_loss: 0.4799 - val_accuracy: 0.8260 - 561ms/epoch - 4ms/step
Epoch 70/150
152/152 - 1s - loss: 0.2207 - accuracy: 0.9085 - val_loss: 0.4706 - val_accuracy: 0.8357 - 511ms/epoch - 3ms/step
Epoch 71/150
152/152 - 1s - loss: 0.2186 - accuracy: 0.9125 - val_loss: 0.5010 - val_accuracy: 0.8390 - 521ms/epoch - 3ms/step
Epoch 72/150
152/152 - 0s - loss: 0.2176 - accuracy: 0.9102 - val_loss: 0.4726 - val_accuracy: 0.8328 - 468ms/epoch - 3ms/step
Epoch 73/150
152/152 - 0s - loss: 0.2158 - accuracy: 0.9112 - val_loss: 0.4697 - val_accuracy: 0.8251 - 480ms/e

Epoch 131/150
152/152 - 0s - loss: 0.1431 - accuracy: 0.9446 - val_loss: 0.7201 - val_accuracy: 0.8025 - 445ms/epoch - 3ms/step
Epoch 132/150
152/152 - 0s - loss: 0.1441 - accuracy: 0.9419 - val_loss: 0.6275 - val_accuracy: 0.8309 - 465ms/epoch - 3ms/step
Epoch 133/150
152/152 - 1s - loss: 0.1414 - accuracy: 0.9452 - val_loss: 0.6544 - val_accuracy: 0.8275 - 591ms/epoch - 4ms/step
Epoch 134/150
152/152 - 1s - loss: 0.1429 - accuracy: 0.9429 - val_loss: 0.6686 - val_accuracy: 0.8318 - 533ms/epoch - 4ms/step
Epoch 135/150
152/152 - 0s - loss: 0.1403 - accuracy: 0.9456 - val_loss: 0.6132 - val_accuracy: 0.8236 - 498ms/epoch - 3ms/step
Epoch 136/150
152/152 - 0s - loss: 0.1400 - accuracy: 0.9469 - val_loss: 0.6429 - val_accuracy: 0.8188 - 486ms/epoch - 3ms/step
Epoch 137/150
152/152 - 0s - loss: 0.1380 - accuracy: 0.9469 - val_loss: 0.7071 - val_accuracy: 0.8328 - 471ms/epoch - 3ms/step
Epoch 138/150
152/152 - 0s - loss: 0.1352 - accuracy: 0.9485 - val_loss: 0.6793 - val_accuracy: 0.8294 -

In [47]:
yhat = model.predict(valid_padded1)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y1_test, yhat))

              precision    recall  f1-score   support

           0       0.83      0.82      0.82      1032
           1       0.82      0.84      0.83      1049

    accuracy                           0.83      2081
   macro avg       0.83      0.83      0.83      2081
weighted avg       0.83      0.83      0.83      2081



In [48]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
t1 = time.time()
history = model.fit(train_padded1, y1_train,
                    validation_data = (valid_padded1, y1_test),
                    epochs = 150,
                    verbose = 2)
t2 = time.time()

print('\nTraining took: {:.2f} minutes'.format((t2 - t1) / 60))

Epoch 1/150
152/152 - 2s - loss: 0.6887 - accuracy: 0.5660 - val_loss: 0.6677 - val_accuracy: 0.7074 - 2s/epoch - 11ms/step
Epoch 2/150
152/152 - 1s - loss: 0.5881 - accuracy: 0.7203 - val_loss: 0.5074 - val_accuracy: 0.7669 - 615ms/epoch - 4ms/step
Epoch 3/150
152/152 - 1s - loss: 0.4715 - accuracy: 0.7817 - val_loss: 0.4601 - val_accuracy: 0.7852 - 708ms/epoch - 5ms/step
Epoch 4/150
152/152 - 1s - loss: 0.3994 - accuracy: 0.8227 - val_loss: 0.4134 - val_accuracy: 0.8241 - 739ms/epoch - 5ms/step
Epoch 5/150
152/152 - 1s - loss: 0.3605 - accuracy: 0.8447 - val_loss: 0.4455 - val_accuracy: 0.7943 - 749ms/epoch - 5ms/step
Epoch 6/150
152/152 - 1s - loss: 0.3349 - accuracy: 0.8509 - val_loss: 0.3989 - val_accuracy: 0.8352 - 746ms/epoch - 5ms/step
Epoch 7/150
152/152 - 1s - loss: 0.3230 - accuracy: 0.8564 - val_loss: 0.4059 - val_accuracy: 0.8275 - 684ms/epoch - 5ms/step
Epoch 8/150
152/152 - 1s - loss: 0.3064 - accuracy: 0.8671 - val_loss: 0.4187 - val_accuracy: 0.8188 - 772ms/epoch - 5ms

Epoch 66/150
152/152 - 1s - loss: 0.1520 - accuracy: 0.9384 - val_loss: 0.7867 - val_accuracy: 0.8121 - 688ms/epoch - 5ms/step
Epoch 67/150
152/152 - 1s - loss: 0.1492 - accuracy: 0.9403 - val_loss: 0.7086 - val_accuracy: 0.8150 - 610ms/epoch - 4ms/step
Epoch 68/150
152/152 - 1s - loss: 0.1435 - accuracy: 0.9460 - val_loss: 0.7286 - val_accuracy: 0.8193 - 588ms/epoch - 4ms/step
Epoch 69/150
152/152 - 1s - loss: 0.1426 - accuracy: 0.9448 - val_loss: 0.7304 - val_accuracy: 0.8198 - 614ms/epoch - 4ms/step
Epoch 70/150
152/152 - 1s - loss: 0.1466 - accuracy: 0.9452 - val_loss: 0.7310 - val_accuracy: 0.8145 - 641ms/epoch - 4ms/step
Epoch 71/150
152/152 - 1s - loss: 0.1536 - accuracy: 0.9429 - val_loss: 0.7250 - val_accuracy: 0.8203 - 635ms/epoch - 4ms/step
Epoch 72/150
152/152 - 1s - loss: 0.1416 - accuracy: 0.9462 - val_loss: 0.7671 - val_accuracy: 0.8203 - 619ms/epoch - 4ms/step
Epoch 73/150
152/152 - 1s - loss: 0.1385 - accuracy: 0.9479 - val_loss: 0.7516 - val_accuracy: 0.8131 - 626ms/e

Epoch 131/150
152/152 - 1s - loss: 0.0660 - accuracy: 0.9718 - val_loss: 1.7148 - val_accuracy: 0.8111 - 577ms/epoch - 4ms/step
Epoch 132/150
152/152 - 1s - loss: 0.0540 - accuracy: 0.9780 - val_loss: 1.8532 - val_accuracy: 0.8217 - 537ms/epoch - 4ms/step
Epoch 133/150
152/152 - 1s - loss: 0.0480 - accuracy: 0.9808 - val_loss: 1.7498 - val_accuracy: 0.8337 - 595ms/epoch - 4ms/step
Epoch 134/150
152/152 - 1s - loss: 0.0453 - accuracy: 0.9813 - val_loss: 1.6657 - val_accuracy: 0.8313 - 549ms/epoch - 4ms/step
Epoch 135/150
152/152 - 1s - loss: 0.0517 - accuracy: 0.9794 - val_loss: 1.7340 - val_accuracy: 0.8304 - 595ms/epoch - 4ms/step
Epoch 136/150
152/152 - 1s - loss: 0.0527 - accuracy: 0.9769 - val_loss: 1.8257 - val_accuracy: 0.8155 - 624ms/epoch - 4ms/step
Epoch 137/150
152/152 - 1s - loss: 0.0502 - accuracy: 0.9804 - val_loss: 2.0121 - val_accuracy: 0.8280 - 672ms/epoch - 4ms/step
Epoch 138/150
152/152 - 1s - loss: 0.0590 - accuracy: 0.9755 - val_loss: 1.8180 - val_accuracy: 0.8256 -

In [49]:
yhat = model.predict(valid_padded1)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y1_test, yhat))

              precision    recall  f1-score   support

           0       0.86      0.77      0.81      1032
           1       0.79      0.88      0.83      1049

    accuracy                           0.82      2081
   macro avg       0.83      0.82      0.82      2081
weighted avg       0.83      0.82      0.82      2081



In [50]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Conv1D(32, 3, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(pool_size = 2, strides = 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(24, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
t1 = time.time()
history = model.fit(train_padded1, y1_train,
                    validation_data = (valid_padded1, y1_test),
                    epochs = 150,
                    verbose = 2)
t2 = time.time()

print('\nTraining took: {:.2f} minutes'.format((t2 - t1) / 60))

Epoch 1/150
152/152 - 4s - loss: 0.6158 - accuracy: 0.6468 - val_loss: 0.4448 - val_accuracy: 0.8015 - 4s/epoch - 23ms/step
Epoch 2/150
152/152 - 3s - loss: 0.3832 - accuracy: 0.8301 - val_loss: 0.3985 - val_accuracy: 0.8265 - 3s/epoch - 18ms/step
Epoch 3/150
152/152 - 3s - loss: 0.2799 - accuracy: 0.8894 - val_loss: 0.4006 - val_accuracy: 0.8260 - 3s/epoch - 18ms/step
Epoch 4/150
152/152 - 3s - loss: 0.2091 - accuracy: 0.9234 - val_loss: 0.4147 - val_accuracy: 0.8385 - 3s/epoch - 18ms/step
Epoch 5/150
152/152 - 3s - loss: 0.1590 - accuracy: 0.9429 - val_loss: 0.4500 - val_accuracy: 0.8462 - 3s/epoch - 19ms/step
Epoch 6/150
152/152 - 3s - loss: 0.1224 - accuracy: 0.9580 - val_loss: 0.4622 - val_accuracy: 0.8510 - 3s/epoch - 19ms/step
Epoch 7/150
152/152 - 3s - loss: 0.1016 - accuracy: 0.9654 - val_loss: 0.4963 - val_accuracy: 0.8486 - 3s/epoch - 18ms/step
Epoch 8/150
152/152 - 3s - loss: 0.0851 - accuracy: 0.9728 - val_loss: 0.5603 - val_accuracy: 0.8424 - 3s/epoch - 19ms/step
Epoch 9/

Epoch 67/150
152/152 - 3s - loss: 0.0258 - accuracy: 0.9889 - val_loss: 1.3993 - val_accuracy: 0.8467 - 3s/epoch - 20ms/step
Epoch 68/150
152/152 - 3s - loss: 0.0251 - accuracy: 0.9891 - val_loss: 1.5337 - val_accuracy: 0.8366 - 3s/epoch - 20ms/step
Epoch 69/150
152/152 - 3s - loss: 0.0249 - accuracy: 0.9874 - val_loss: 1.4688 - val_accuracy: 0.8587 - 3s/epoch - 19ms/step
Epoch 70/150
152/152 - 3s - loss: 0.0257 - accuracy: 0.9876 - val_loss: 1.6380 - val_accuracy: 0.8448 - 3s/epoch - 20ms/step
Epoch 71/150
152/152 - 3s - loss: 0.0303 - accuracy: 0.9862 - val_loss: 1.3720 - val_accuracy: 0.8515 - 3s/epoch - 19ms/step
Epoch 72/150
152/152 - 3s - loss: 0.0248 - accuracy: 0.9883 - val_loss: 1.4691 - val_accuracy: 0.8525 - 3s/epoch - 18ms/step
Epoch 73/150
152/152 - 3s - loss: 0.0236 - accuracy: 0.9893 - val_loss: 1.4868 - val_accuracy: 0.8501 - 3s/epoch - 19ms/step
Epoch 74/150
152/152 - 3s - loss: 0.0248 - accuracy: 0.9893 - val_loss: 1.4660 - val_accuracy: 0.8496 - 3s/epoch - 19ms/step


Epoch 133/150
152/152 - 3s - loss: 0.0223 - accuracy: 0.9891 - val_loss: 1.6206 - val_accuracy: 0.8429 - 3s/epoch - 21ms/step
Epoch 134/150
152/152 - 3s - loss: 0.0230 - accuracy: 0.9899 - val_loss: 1.6685 - val_accuracy: 0.8357 - 3s/epoch - 20ms/step
Epoch 135/150
152/152 - 3s - loss: 0.0234 - accuracy: 0.9883 - val_loss: 1.6216 - val_accuracy: 0.8405 - 3s/epoch - 21ms/step
Epoch 136/150
152/152 - 3s - loss: 0.0227 - accuracy: 0.9885 - val_loss: 1.6471 - val_accuracy: 0.8409 - 3s/epoch - 22ms/step
Epoch 137/150
152/152 - 3s - loss: 0.0209 - accuracy: 0.9891 - val_loss: 1.7402 - val_accuracy: 0.8438 - 3s/epoch - 21ms/step
Epoch 138/150
152/152 - 3s - loss: 0.0230 - accuracy: 0.9887 - val_loss: 1.7142 - val_accuracy: 0.8361 - 3s/epoch - 21ms/step
Epoch 139/150
152/152 - 3s - loss: 0.0186 - accuracy: 0.9905 - val_loss: 1.7735 - val_accuracy: 0.8486 - 3s/epoch - 21ms/step
Epoch 140/150
152/152 - 3s - loss: 0.0203 - accuracy: 0.9909 - val_loss: 1.7537 - val_accuracy: 0.8582 - 3s/epoch - 21

In [51]:
yhat = model.predict(valid_padded1)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y1_test, yhat))

              precision    recall  f1-score   support

           0       0.87      0.77      0.82      1032
           1       0.79      0.89      0.84      1049

    accuracy                           0.83      2081
   macro avg       0.83      0.83      0.83      2081
weighted avg       0.83      0.83      0.83      2081



In [52]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Conv1D(32, 3, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(pool_size = 2, strides = 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(24, activation = 'elu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Dense(16, activation = 'elu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
t1 = time.time()
history = model.fit(train_padded1, y1_train,
                    validation_data = (valid_padded1, y1_test),
                    epochs = 150,
                    verbose = 2)
t2 = time.time()

print('\nTraining took: {:.2f} minutes'.format((t2 - t1) / 60))

Epoch 1/150
152/152 - 3s - loss: 0.5689 - accuracy: 0.6871 - val_loss: 0.4183 - val_accuracy: 0.8198 - 3s/epoch - 19ms/step
Epoch 2/150
152/152 - 2s - loss: 0.3401 - accuracy: 0.8552 - val_loss: 0.3732 - val_accuracy: 0.8409 - 2s/epoch - 14ms/step
Epoch 3/150
152/152 - 2s - loss: 0.2181 - accuracy: 0.9191 - val_loss: 0.3815 - val_accuracy: 0.8477 - 2s/epoch - 14ms/step
Epoch 4/150
152/152 - 2s - loss: 0.1539 - accuracy: 0.9491 - val_loss: 0.4311 - val_accuracy: 0.8491 - 2s/epoch - 13ms/step
Epoch 5/150
152/152 - 2s - loss: 0.1178 - accuracy: 0.9594 - val_loss: 0.4720 - val_accuracy: 0.8486 - 2s/epoch - 14ms/step
Epoch 6/150
152/152 - 2s - loss: 0.0972 - accuracy: 0.9691 - val_loss: 0.5052 - val_accuracy: 0.8443 - 2s/epoch - 14ms/step
Epoch 7/150
152/152 - 2s - loss: 0.0877 - accuracy: 0.9710 - val_loss: 0.5006 - val_accuracy: 0.8568 - 2s/epoch - 14ms/step
Epoch 8/150
152/152 - 2s - loss: 0.0777 - accuracy: 0.9743 - val_loss: 0.5630 - val_accuracy: 0.8328 - 2s/epoch - 14ms/step
Epoch 9/

Epoch 67/150
152/152 - 3s - loss: 0.0240 - accuracy: 0.9874 - val_loss: 1.1476 - val_accuracy: 0.8544 - 3s/epoch - 21ms/step
Epoch 68/150
152/152 - 3s - loss: 0.0325 - accuracy: 0.9864 - val_loss: 1.1229 - val_accuracy: 0.8438 - 3s/epoch - 22ms/step
Epoch 69/150
152/152 - 3s - loss: 0.0246 - accuracy: 0.9876 - val_loss: 1.2212 - val_accuracy: 0.8337 - 3s/epoch - 22ms/step
Epoch 70/150
152/152 - 3s - loss: 0.0251 - accuracy: 0.9866 - val_loss: 1.2121 - val_accuracy: 0.8515 - 3s/epoch - 22ms/step
Epoch 71/150
152/152 - 3s - loss: 0.0270 - accuracy: 0.9874 - val_loss: 1.2075 - val_accuracy: 0.8453 - 3s/epoch - 22ms/step
Epoch 72/150
152/152 - 3s - loss: 0.0244 - accuracy: 0.9874 - val_loss: 1.3155 - val_accuracy: 0.8414 - 3s/epoch - 23ms/step
Epoch 73/150
152/152 - 4s - loss: 0.0287 - accuracy: 0.9874 - val_loss: 1.1807 - val_accuracy: 0.8381 - 4s/epoch - 24ms/step
Epoch 74/150
152/152 - 4s - loss: 0.0235 - accuracy: 0.9881 - val_loss: 1.2967 - val_accuracy: 0.8275 - 4s/epoch - 23ms/step


Epoch 133/150
152/152 - 3s - loss: 0.0211 - accuracy: 0.9887 - val_loss: 1.3508 - val_accuracy: 0.8453 - 3s/epoch - 21ms/step
Epoch 134/150
152/152 - 3s - loss: 0.0210 - accuracy: 0.9883 - val_loss: 1.4105 - val_accuracy: 0.8491 - 3s/epoch - 22ms/step
Epoch 135/150
152/152 - 3s - loss: 0.0203 - accuracy: 0.9903 - val_loss: 1.4601 - val_accuracy: 0.8506 - 3s/epoch - 22ms/step
Epoch 136/150
152/152 - 3s - loss: 0.0262 - accuracy: 0.9878 - val_loss: 1.2465 - val_accuracy: 0.8510 - 3s/epoch - 21ms/step
Epoch 137/150
152/152 - 3s - loss: 0.0209 - accuracy: 0.9903 - val_loss: 1.3062 - val_accuracy: 0.8400 - 3s/epoch - 22ms/step
Epoch 138/150
152/152 - 3s - loss: 0.0219 - accuracy: 0.9883 - val_loss: 1.2952 - val_accuracy: 0.8544 - 3s/epoch - 22ms/step
Epoch 139/150
152/152 - 3s - loss: 0.0209 - accuracy: 0.9881 - val_loss: 1.4993 - val_accuracy: 0.8309 - 3s/epoch - 22ms/step
Epoch 140/150
152/152 - 3s - loss: 0.0225 - accuracy: 0.9889 - val_loss: 1.3566 - val_accuracy: 0.8390 - 3s/epoch - 22

In [53]:
yhat = model.predict(valid_padded1)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y1_test, yhat))

              precision    recall  f1-score   support

           0       0.87      0.79      0.83      1032
           1       0.81      0.88      0.85      1049

    accuracy                           0.84      2081
   macro avg       0.84      0.84      0.84      2081
weighted avg       0.84      0.84      0.84      2081



In [54]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(24, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
t1 = time.time()
history = model.fit(train_padded1, y1_train,
                    validation_data = (valid_padded1, y1_test),
                    epochs = 150,
                    verbose = 2)
t2 = time.time()

print('\nTraining took: {:.2f} minutes'.format((t2 - t1) / 60))

Epoch 1/150
152/152 - 16s - loss: 0.6934 - accuracy: 0.5020 - val_loss: 0.6931 - val_accuracy: 0.5041 - 16s/epoch - 103ms/step
Epoch 2/150
152/152 - 11s - loss: 0.6932 - accuracy: 0.5055 - val_loss: 0.6952 - val_accuracy: 0.4959 - 11s/epoch - 70ms/step
Epoch 3/150
152/152 - 11s - loss: 0.6937 - accuracy: 0.4982 - val_loss: 0.6931 - val_accuracy: 0.5041 - 11s/epoch - 72ms/step
Epoch 4/150
152/152 - 13s - loss: 0.6935 - accuracy: 0.4941 - val_loss: 0.6931 - val_accuracy: 0.5041 - 13s/epoch - 86ms/step
Epoch 5/150
152/152 - 13s - loss: 0.6935 - accuracy: 0.4991 - val_loss: 0.6935 - val_accuracy: 0.5041 - 13s/epoch - 86ms/step
Epoch 6/150
152/152 - 7s - loss: 0.6936 - accuracy: 0.4859 - val_loss: 0.6931 - val_accuracy: 0.5041 - 7s/epoch - 44ms/step
Epoch 7/150
152/152 - 6s - loss: 0.6933 - accuracy: 0.4987 - val_loss: 0.6931 - val_accuracy: 0.5041 - 6s/epoch - 42ms/step
Epoch 8/150


KeyboardInterrupt: 

In [55]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(512, activation = 'elu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
t1 = time.time()
history = model.fit(train_padded1, y1_train,
                    validation_data = (valid_padded1, y1_test),
                    epochs = 150,
                    verbose = 2)
t2 = time.time()

print('\nTraining took: {:.2f} minutes'.format((t2 - t1) / 60))

Epoch 1/150
152/152 - 1s - loss: 0.6745 - accuracy: 0.5708 - val_loss: 0.6009 - val_accuracy: 0.6593 - 806ms/epoch - 5ms/step
Epoch 2/150
152/152 - 0s - loss: 0.4968 - accuracy: 0.7561 - val_loss: 0.4602 - val_accuracy: 0.7833 - 351ms/epoch - 2ms/step
Epoch 3/150
152/152 - 0s - loss: 0.3947 - accuracy: 0.8264 - val_loss: 0.4094 - val_accuracy: 0.8188 - 352ms/epoch - 2ms/step
Epoch 4/150
152/152 - 0s - loss: 0.3433 - accuracy: 0.8468 - val_loss: 0.4041 - val_accuracy: 0.8337 - 355ms/epoch - 2ms/step
Epoch 5/150
152/152 - 0s - loss: 0.3088 - accuracy: 0.8680 - val_loss: 0.4399 - val_accuracy: 0.8155 - 354ms/epoch - 2ms/step
Epoch 6/150
152/152 - 0s - loss: 0.2963 - accuracy: 0.8742 - val_loss: 0.4539 - val_accuracy: 0.8063 - 447ms/epoch - 3ms/step
Epoch 7/150
152/152 - 0s - loss: 0.2853 - accuracy: 0.8748 - val_loss: 0.4413 - val_accuracy: 0.8193 - 361ms/epoch - 2ms/step
Epoch 8/150
152/152 - 0s - loss: 0.2714 - accuracy: 0.8828 - val_loss: 0.4293 - val_accuracy: 0.8294 - 400ms/epoch - 3

Epoch 66/150
152/152 - 0s - loss: 0.1535 - accuracy: 0.9384 - val_loss: 1.0519 - val_accuracy: 0.8011 - 440ms/epoch - 3ms/step
Epoch 67/150
152/152 - 0s - loss: 0.1538 - accuracy: 0.9382 - val_loss: 0.9105 - val_accuracy: 0.8131 - 434ms/epoch - 3ms/step
Epoch 68/150
152/152 - 0s - loss: 0.1455 - accuracy: 0.9425 - val_loss: 0.9469 - val_accuracy: 0.8111 - 367ms/epoch - 2ms/step
Epoch 69/150
152/152 - 0s - loss: 0.1481 - accuracy: 0.9429 - val_loss: 0.9633 - val_accuracy: 0.8203 - 378ms/epoch - 2ms/step
Epoch 70/150
152/152 - 0s - loss: 0.1507 - accuracy: 0.9384 - val_loss: 0.9167 - val_accuracy: 0.8131 - 411ms/epoch - 3ms/step
Epoch 71/150
152/152 - 0s - loss: 0.1602 - accuracy: 0.9368 - val_loss: 0.8815 - val_accuracy: 0.8193 - 361ms/epoch - 2ms/step
Epoch 72/150
152/152 - 0s - loss: 0.1467 - accuracy: 0.9425 - val_loss: 0.9804 - val_accuracy: 0.8203 - 437ms/epoch - 3ms/step
Epoch 73/150
152/152 - 0s - loss: 0.1469 - accuracy: 0.9427 - val_loss: 0.9806 - val_accuracy: 0.8107 - 426ms/e

Epoch 131/150
152/152 - 0s - loss: 0.1249 - accuracy: 0.9485 - val_loss: 1.4445 - val_accuracy: 0.8126 - 352ms/epoch - 2ms/step
Epoch 132/150
152/152 - 0s - loss: 0.1309 - accuracy: 0.9475 - val_loss: 1.2616 - val_accuracy: 0.8116 - 447ms/epoch - 3ms/step
Epoch 133/150
152/152 - 0s - loss: 0.1254 - accuracy: 0.9532 - val_loss: 1.2914 - val_accuracy: 0.8126 - 378ms/epoch - 2ms/step
Epoch 134/150
152/152 - 0s - loss: 0.1289 - accuracy: 0.9481 - val_loss: 1.3869 - val_accuracy: 0.8140 - 459ms/epoch - 3ms/step
Epoch 135/150
152/152 - 0s - loss: 0.1258 - accuracy: 0.9516 - val_loss: 1.3066 - val_accuracy: 0.8155 - 399ms/epoch - 3ms/step
Epoch 136/150
152/152 - 0s - loss: 0.1332 - accuracy: 0.9446 - val_loss: 1.4695 - val_accuracy: 0.8044 - 439ms/epoch - 3ms/step
Epoch 137/150
152/152 - 0s - loss: 0.1230 - accuracy: 0.9520 - val_loss: 1.3161 - val_accuracy: 0.8107 - 355ms/epoch - 2ms/step
Epoch 138/150
152/152 - 0s - loss: 0.1322 - accuracy: 0.9483 - val_loss: 1.2925 - val_accuracy: 0.8140 -

In [153]:
yhat = model.predict(valid_padded1)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y1_test, yhat))

              precision    recall  f1-score   support

           0       0.85      0.73      0.79      1305
           1       0.76      0.87      0.81      1301

    accuracy                           0.80      2606
   macro avg       0.81      0.80      0.80      2606
weighted avg       0.81      0.80      0.80      2606



In [56]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Conv1D(64, 4, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(pool_size = 2, strides = 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(24, activation = 'elu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Dense(16, activation = 'elu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
t1 = time.time()
history = model.fit(train_padded1, y1_train,
                    validation_data = (valid_padded1, y1_test),
                    epochs = 150,
                    verbose = 2)
t2 = time.time()

print('\nTraining took: {:.2f} minutes'.format((t2 - t1) / 60))

Epoch 1/150
152/152 - 6s - loss: 0.5621 - accuracy: 0.6929 - val_loss: 0.4363 - val_accuracy: 0.8145 - 6s/epoch - 40ms/step
Epoch 2/150
152/152 - 4s - loss: 0.3324 - accuracy: 0.8610 - val_loss: 0.3793 - val_accuracy: 0.8448 - 4s/epoch - 28ms/step
Epoch 3/150
152/152 - 4s - loss: 0.2069 - accuracy: 0.9250 - val_loss: 0.3918 - val_accuracy: 0.8472 - 4s/epoch - 28ms/step
Epoch 4/150
152/152 - 4s - loss: 0.1387 - accuracy: 0.9545 - val_loss: 0.4626 - val_accuracy: 0.8429 - 4s/epoch - 29ms/step
Epoch 5/150
152/152 - 4s - loss: 0.1051 - accuracy: 0.9677 - val_loss: 0.4422 - val_accuracy: 0.8563 - 4s/epoch - 29ms/step
Epoch 6/150
152/152 - 4s - loss: 0.0887 - accuracy: 0.9730 - val_loss: 0.4686 - val_accuracy: 0.8602 - 4s/epoch - 29ms/step
Epoch 7/150
152/152 - 4s - loss: 0.0637 - accuracy: 0.9786 - val_loss: 0.4924 - val_accuracy: 0.8587 - 4s/epoch - 29ms/step
Epoch 8/150
152/152 - 4s - loss: 0.0596 - accuracy: 0.9792 - val_loss: 0.5642 - val_accuracy: 0.8405 - 4s/epoch - 29ms/step
Epoch 9/

Epoch 67/150
152/152 - 5s - loss: 0.0240 - accuracy: 0.9883 - val_loss: 1.2755 - val_accuracy: 0.8549 - 5s/epoch - 30ms/step
Epoch 68/150
152/152 - 5s - loss: 0.0238 - accuracy: 0.9891 - val_loss: 1.2115 - val_accuracy: 0.8539 - 5s/epoch - 30ms/step
Epoch 69/150
152/152 - 5s - loss: 0.0204 - accuracy: 0.9895 - val_loss: 1.4425 - val_accuracy: 0.8477 - 5s/epoch - 30ms/step
Epoch 70/150
152/152 - 5s - loss: 0.0278 - accuracy: 0.9862 - val_loss: 1.1739 - val_accuracy: 0.8510 - 5s/epoch - 30ms/step
Epoch 71/150
152/152 - 5s - loss: 0.0238 - accuracy: 0.9883 - val_loss: 1.2410 - val_accuracy: 0.8419 - 5s/epoch - 30ms/step
Epoch 72/150
152/152 - 5s - loss: 0.0226 - accuracy: 0.9878 - val_loss: 1.2374 - val_accuracy: 0.8371 - 5s/epoch - 30ms/step
Epoch 73/150
152/152 - 4s - loss: 0.0243 - accuracy: 0.9885 - val_loss: 1.1331 - val_accuracy: 0.8462 - 4s/epoch - 30ms/step
Epoch 74/150
152/152 - 5s - loss: 0.0234 - accuracy: 0.9874 - val_loss: 1.2469 - val_accuracy: 0.8390 - 5s/epoch - 30ms/step


Epoch 133/150
152/152 - 5s - loss: 0.0201 - accuracy: 0.9883 - val_loss: 1.3980 - val_accuracy: 0.8371 - 5s/epoch - 31ms/step
Epoch 134/150
152/152 - 5s - loss: 0.0216 - accuracy: 0.9897 - val_loss: 1.4285 - val_accuracy: 0.8227 - 5s/epoch - 30ms/step
Epoch 135/150
152/152 - 5s - loss: 0.0238 - accuracy: 0.9891 - val_loss: 1.2960 - val_accuracy: 0.8313 - 5s/epoch - 30ms/step
Epoch 136/150
152/152 - 4s - loss: 0.0215 - accuracy: 0.9891 - val_loss: 1.1873 - val_accuracy: 0.8419 - 4s/epoch - 29ms/step
Epoch 137/150
152/152 - 5s - loss: 0.0195 - accuracy: 0.9885 - val_loss: 1.2430 - val_accuracy: 0.8419 - 5s/epoch - 33ms/step
Epoch 138/150
152/152 - 4s - loss: 0.0186 - accuracy: 0.9897 - val_loss: 1.3313 - val_accuracy: 0.8419 - 4s/epoch - 28ms/step
Epoch 139/150
152/152 - 4s - loss: 0.0207 - accuracy: 0.9895 - val_loss: 1.2811 - val_accuracy: 0.8337 - 4s/epoch - 28ms/step
Epoch 140/150
152/152 - 4s - loss: 0.0190 - accuracy: 0.9897 - val_loss: 1.4216 - val_accuracy: 0.8385 - 4s/epoch - 28

In [155]:
yhat = model.predict(valid_padded1)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y1_test, yhat))

              precision    recall  f1-score   support

           0       0.85      0.83      0.84      1305
           1       0.83      0.85      0.84      1301

    accuracy                           0.84      2606
   macro avg       0.84      0.84      0.84      2606
weighted avg       0.84      0.84      0.84      2606



In [57]:
model.save('NLP_raw_tweets.h5')
print('Saved!!!')

Saved!!!


In [68]:
vocab_size = 1500
max_length = 120
embedding_dim = 512
oov_tok = '<OOV>'
padding_type = 'post'
trunc_type = 'post'

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(X1_train)
word_index = tokenizer.word_index

train_sequence1 = tokenizer.texts_to_sequences(X1_train)
train_padded1 = pad_sequences(train_sequence1, maxlen = max_length, padding = padding_type, truncating = trunc_type)

valid_sequence1 = tokenizer.texts_to_sequences(X1_test)
valid_padded1 = pad_sequences(valid_sequence1, maxlen = max_length, padding = padding_type, truncating = trunc_type)

tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Conv1D(128, 6, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(pool_size = 2, strides = 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(300, activation = 'elu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Dense(100, activation = 'elu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
t1 = time.time()
history = model.fit(train_padded1, y1_train,
                    validation_data = (valid_padded1, y1_test),
                    epochs = 100,
                    verbose = 2)
t2 = time.time()

print('\nTraining took: {:.2f} minutes'.format((t2 - t1) / 60))

Epoch 1/100
152/152 - 12s - loss: 0.5470 - accuracy: 0.7217 - val_loss: 0.4056 - val_accuracy: 0.8232 - 12s/epoch - 77ms/step
Epoch 2/100
152/152 - 11s - loss: 0.2930 - accuracy: 0.8867 - val_loss: 0.3777 - val_accuracy: 0.8424 - 11s/epoch - 72ms/step
Epoch 3/100
152/152 - 11s - loss: 0.1623 - accuracy: 0.9432 - val_loss: 0.4855 - val_accuracy: 0.8501 - 11s/epoch - 72ms/step
Epoch 4/100
152/152 - 11s - loss: 0.0981 - accuracy: 0.9658 - val_loss: 0.5334 - val_accuracy: 0.8525 - 11s/epoch - 72ms/step
Epoch 5/100
152/152 - 11s - loss: 0.0689 - accuracy: 0.9749 - val_loss: 0.6983 - val_accuracy: 0.8530 - 11s/epoch - 72ms/step
Epoch 6/100
152/152 - 11s - loss: 0.0760 - accuracy: 0.9708 - val_loss: 0.6671 - val_accuracy: 0.8496 - 11s/epoch - 72ms/step
Epoch 7/100
152/152 - 11s - loss: 0.0746 - accuracy: 0.9701 - val_loss: 0.6379 - val_accuracy: 0.8361 - 11s/epoch - 71ms/step
Epoch 8/100
152/152 - 11s - loss: 0.0631 - accuracy: 0.9743 - val_loss: 0.7610 - val_accuracy: 0.8405 - 11s/epoch - 73

Epoch 66/100
152/152 - 11s - loss: 0.0265 - accuracy: 0.9870 - val_loss: 1.2399 - val_accuracy: 0.8385 - 11s/epoch - 72ms/step
Epoch 67/100
152/152 - 11s - loss: 0.0228 - accuracy: 0.9883 - val_loss: 1.2845 - val_accuracy: 0.8453 - 11s/epoch - 73ms/step
Epoch 68/100
152/152 - 11s - loss: 0.0226 - accuracy: 0.9881 - val_loss: 1.4516 - val_accuracy: 0.8438 - 11s/epoch - 74ms/step
Epoch 69/100
152/152 - 11s - loss: 0.0269 - accuracy: 0.9891 - val_loss: 0.9046 - val_accuracy: 0.8366 - 11s/epoch - 74ms/step
Epoch 70/100
152/152 - 11s - loss: 0.0241 - accuracy: 0.9881 - val_loss: 1.6247 - val_accuracy: 0.8429 - 11s/epoch - 74ms/step
Epoch 71/100
152/152 - 11s - loss: 0.0234 - accuracy: 0.9883 - val_loss: 1.6445 - val_accuracy: 0.8222 - 11s/epoch - 74ms/step
Epoch 72/100
152/152 - 11s - loss: 0.0297 - accuracy: 0.9864 - val_loss: 1.0506 - val_accuracy: 0.8347 - 11s/epoch - 74ms/step
Epoch 73/100
152/152 - 11s - loss: 0.0226 - accuracy: 0.9885 - val_loss: 1.2421 - val_accuracy: 0.8390 - 11s/ep

In [69]:
yhat = model.predict(valid_padded1)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y1_test, yhat))

              precision    recall  f1-score   support

           0       0.86      0.79      0.83      1032
           1       0.81      0.88      0.84      1049

    accuracy                           0.83      2081
   macro avg       0.84      0.83      0.83      2081
weighted avg       0.84      0.83      0.83      2081



In [60]:
from tensorflow.keras.models import load_model

df_test = pd.read_csv('test.csv')
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [61]:
X_predict = df_test.iloc[:, -1]

new_sequences = tokenizer.texts_to_sequences(X_predict)
new_padded = pad_sequences(new_sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type)

In [62]:
raw_model = load_model('NLP_raw_tweets.h5')

In [63]:
y_predict = raw_model.predict(new_padded)
y_predict = np.round(y_predict).reshape(-1)
y_predict

array([0., 1., 1., ..., 1., 0., 0.], dtype=float32)

In [64]:
df_test['target'] = y_predict.astype(int)
df_test.head()

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,0
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,0
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1


In [65]:
submission = df_test.iloc[:, [0, -1]]
submission.set_index('id', inplace=True)
submission

,target
id,
0,0
2,1
3,1
9,0
11,1
...,...
10861,1
10865,0
10868,1


In [66]:
submission.to_csv('submission_raw.csv')

In [156]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
t1 = time.time()
history = model.fit(train_padded1, y1_train,
                    validation_data = (valid_padded1, y1_test),
                    epochs = 150,
                    verbose = 2)
t2 = time.time()

print('\nTraining took: {:.2f} minutes'.format((t2 - t1) / 60))

Epoch 1/150
190/190 - 13s - loss: 0.5403 - accuracy: 0.7244 - val_loss: 0.4395 - val_accuracy: 0.7924 - 13s/epoch - 66ms/step
Epoch 2/150
190/190 - 7s - loss: 0.3871 - accuracy: 0.8309 - val_loss: 0.4302 - val_accuracy: 0.8112 - 7s/epoch - 37ms/step
Epoch 3/150
190/190 - 7s - loss: 0.3405 - accuracy: 0.8595 - val_loss: 0.4342 - val_accuracy: 0.8139 - 7s/epoch - 39ms/step
Epoch 4/150
190/190 - 7s - loss: 0.3078 - accuracy: 0.8720 - val_loss: 0.4527 - val_accuracy: 0.8208 - 7s/epoch - 39ms/step
Epoch 5/150
190/190 - 7s - loss: 0.2718 - accuracy: 0.8975 - val_loss: 0.4462 - val_accuracy: 0.8185 - 7s/epoch - 39ms/step
Epoch 6/150
190/190 - 8s - loss: 0.2392 - accuracy: 0.9131 - val_loss: 0.4560 - val_accuracy: 0.8127 - 8s/epoch - 40ms/step
Epoch 7/150
190/190 - 8s - loss: 0.2183 - accuracy: 0.9181 - val_loss: 0.5412 - val_accuracy: 0.8124 - 8s/epoch - 41ms/step
Epoch 8/150
190/190 - 8s - loss: 0.1927 - accuracy: 0.9307 - val_loss: 0.5107 - val_accuracy: 0.8235 - 8s/epoch - 41ms/step
Epoch 

Epoch 67/150
190/190 - 9s - loss: 0.0294 - accuracy: 0.9849 - val_loss: 1.5214 - val_accuracy: 0.8173 - 9s/epoch - 45ms/step
Epoch 68/150
190/190 - 9s - loss: 0.0343 - accuracy: 0.9844 - val_loss: 1.4496 - val_accuracy: 0.8219 - 9s/epoch - 46ms/step
Epoch 69/150
190/190 - 9s - loss: 0.0285 - accuracy: 0.9855 - val_loss: 1.5020 - val_accuracy: 0.8415 - 9s/epoch - 46ms/step
Epoch 70/150
190/190 - 9s - loss: 0.0334 - accuracy: 0.9840 - val_loss: 1.5437 - val_accuracy: 0.8227 - 9s/epoch - 48ms/step
Epoch 71/150
190/190 - 9s - loss: 0.0348 - accuracy: 0.9831 - val_loss: 1.4601 - val_accuracy: 0.8120 - 9s/epoch - 48ms/step
Epoch 72/150
190/190 - 9s - loss: 0.0376 - accuracy: 0.9812 - val_loss: 1.4719 - val_accuracy: 0.8166 - 9s/epoch - 48ms/step
Epoch 73/150
190/190 - 9s - loss: 0.0281 - accuracy: 0.9859 - val_loss: 1.5886 - val_accuracy: 0.8219 - 9s/epoch - 46ms/step
Epoch 74/150
190/190 - 9s - loss: 0.0260 - accuracy: 0.9855 - val_loss: 1.6510 - val_accuracy: 0.8150 - 9s/epoch - 46ms/step


Epoch 133/150
190/190 - 9s - loss: 0.0218 - accuracy: 0.9867 - val_loss: 1.8506 - val_accuracy: 0.8281 - 9s/epoch - 46ms/step
Epoch 134/150
190/190 - 9s - loss: 0.0218 - accuracy: 0.9872 - val_loss: 1.8600 - val_accuracy: 0.8308 - 9s/epoch - 46ms/step
Epoch 135/150
190/190 - 9s - loss: 0.0219 - accuracy: 0.9873 - val_loss: 1.8971 - val_accuracy: 0.8308 - 9s/epoch - 47ms/step
Epoch 136/150
190/190 - 9s - loss: 0.0220 - accuracy: 0.9867 - val_loss: 1.8714 - val_accuracy: 0.8308 - 9s/epoch - 47ms/step
Epoch 137/150
190/190 - 9s - loss: 0.0220 - accuracy: 0.9873 - val_loss: 1.9030 - val_accuracy: 0.8300 - 9s/epoch - 46ms/step
Epoch 138/150
190/190 - 9s - loss: 0.0218 - accuracy: 0.9870 - val_loss: 1.9613 - val_accuracy: 0.8292 - 9s/epoch - 46ms/step
Epoch 139/150
190/190 - 9s - loss: 0.0221 - accuracy: 0.9863 - val_loss: 1.9180 - val_accuracy: 0.8289 - 9s/epoch - 47ms/step
Epoch 140/150
190/190 - 10s - loss: 0.0227 - accuracy: 0.9868 - val_loss: 1.9064 - val_accuracy: 0.8304 - 10s/epoch - 

In [157]:
yhat = model.predict(valid_padded1)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y1_test, yhat))

              precision    recall  f1-score   support

           0       0.85      0.80      0.83      1305
           1       0.81      0.86      0.84      1301

    accuracy                           0.83      2606
   macro avg       0.83      0.83      0.83      2606
weighted avg       0.83      0.83      0.83      2606



In [160]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size = 3, strides = 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
t1 = time.time()
history = model.fit(train_padded1, y1_train,
                    validation_data = (valid_padded1, y1_test),
                    epochs = 150,
                    verbose = 2)
t2 = time.time()

print('\nTraining took: {:.2f} minutes'.format((t2 - t1) / 60))

Epoch 1/150
190/190 - 4s - loss: 0.6314 - accuracy: 0.6677 - val_loss: 0.4985 - val_accuracy: 0.7732 - 4s/epoch - 20ms/step
Epoch 2/150
190/190 - 3s - loss: 0.3954 - accuracy: 0.8277 - val_loss: 0.4117 - val_accuracy: 0.8235 - 3s/epoch - 17ms/step
Epoch 3/150
190/190 - 3s - loss: 0.2989 - accuracy: 0.8802 - val_loss: 0.4114 - val_accuracy: 0.8227 - 3s/epoch - 17ms/step
Epoch 4/150
190/190 - 3s - loss: 0.2273 - accuracy: 0.9112 - val_loss: 0.4369 - val_accuracy: 0.8335 - 3s/epoch - 17ms/step
Epoch 5/150
190/190 - 3s - loss: 0.1649 - accuracy: 0.9408 - val_loss: 0.4589 - val_accuracy: 0.8358 - 3s/epoch - 17ms/step
Epoch 6/150
190/190 - 3s - loss: 0.1182 - accuracy: 0.9589 - val_loss: 0.5039 - val_accuracy: 0.8480 - 3s/epoch - 18ms/step
Epoch 7/150
190/190 - 3s - loss: 0.0922 - accuracy: 0.9694 - val_loss: 0.5558 - val_accuracy: 0.8304 - 3s/epoch - 17ms/step
Epoch 8/150
190/190 - 3s - loss: 0.0748 - accuracy: 0.9742 - val_loss: 0.5786 - val_accuracy: 0.8361 - 3s/epoch - 17ms/step
Epoch 9/

Epoch 67/150
190/190 - 3s - loss: 0.0234 - accuracy: 0.9875 - val_loss: 1.6043 - val_accuracy: 0.8262 - 3s/epoch - 18ms/step
Epoch 68/150
190/190 - 3s - loss: 0.0235 - accuracy: 0.9870 - val_loss: 1.6281 - val_accuracy: 0.8323 - 3s/epoch - 17ms/step
Epoch 69/150
190/190 - 3s - loss: 0.0221 - accuracy: 0.9868 - val_loss: 1.7052 - val_accuracy: 0.8304 - 3s/epoch - 17ms/step
Epoch 70/150
190/190 - 3s - loss: 0.0219 - accuracy: 0.9872 - val_loss: 1.7330 - val_accuracy: 0.8254 - 3s/epoch - 16ms/step
Epoch 71/150
190/190 - 3s - loss: 0.0219 - accuracy: 0.9870 - val_loss: 1.7433 - val_accuracy: 0.8323 - 3s/epoch - 16ms/step
Epoch 72/150
190/190 - 3s - loss: 0.0221 - accuracy: 0.9867 - val_loss: 1.7496 - val_accuracy: 0.8250 - 3s/epoch - 17ms/step
Epoch 73/150
190/190 - 3s - loss: 0.0222 - accuracy: 0.9870 - val_loss: 1.8064 - val_accuracy: 0.8231 - 3s/epoch - 17ms/step
Epoch 74/150
190/190 - 3s - loss: 0.0223 - accuracy: 0.9863 - val_loss: 1.7983 - val_accuracy: 0.8181 - 3s/epoch - 16ms/step


Epoch 133/150
190/190 - 3s - loss: 0.0216 - accuracy: 0.9875 - val_loss: 2.0089 - val_accuracy: 0.8308 - 3s/epoch - 17ms/step
Epoch 134/150
190/190 - 3s - loss: 0.0216 - accuracy: 0.9870 - val_loss: 2.0062 - val_accuracy: 0.8354 - 3s/epoch - 16ms/step
Epoch 135/150
190/190 - 3s - loss: 0.0214 - accuracy: 0.9873 - val_loss: 2.0093 - val_accuracy: 0.8350 - 3s/epoch - 16ms/step
Epoch 136/150
190/190 - 3s - loss: 0.0261 - accuracy: 0.9870 - val_loss: 1.9008 - val_accuracy: 0.8223 - 3s/epoch - 16ms/step
Epoch 137/150
190/190 - 3s - loss: 0.0235 - accuracy: 0.9868 - val_loss: 1.9424 - val_accuracy: 0.8392 - 3s/epoch - 16ms/step
Epoch 138/150
190/190 - 3s - loss: 0.0222 - accuracy: 0.9867 - val_loss: 2.0881 - val_accuracy: 0.8266 - 3s/epoch - 16ms/step
Epoch 139/150
190/190 - 3s - loss: 0.0231 - accuracy: 0.9872 - val_loss: 1.9916 - val_accuracy: 0.8292 - 3s/epoch - 16ms/step
Epoch 140/150
190/190 - 3s - loss: 0.0217 - accuracy: 0.9875 - val_loss: 2.1530 - val_accuracy: 0.8400 - 3s/epoch - 17

In [161]:
yhat = model.predict(valid_padded1)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y1_test, yhat))

              precision    recall  f1-score   support

           0       0.88      0.72      0.79      1305
           1       0.76      0.90      0.82      1301

    accuracy                           0.81      2606
   macro avg       0.82      0.81      0.81      2606
weighted avg       0.82      0.81      0.81      2606



In [162]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Conv1D(32, 3, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(pool_size = 2, strides = 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(24, activation = 'elu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Dense(16, activation = 'elu', kernel_initializer = 'he_normal'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
t1 = time.time()
history = model.fit(train_padded1, y1_train,
                    validation_data = (valid_padded1, y1_test),
                    epochs = 150,
                    verbose = 2)
t2 = time.time()

print('\nTraining took: {:.2f} minutes'.format((t2 - t1) / 60))

Epoch 1/150
190/190 - 3s - loss: 0.5624 - accuracy: 0.6945 - val_loss: 0.4424 - val_accuracy: 0.7920 - 3s/epoch - 13ms/step
Epoch 2/150
190/190 - 2s - loss: 0.3698 - accuracy: 0.8409 - val_loss: 0.4006 - val_accuracy: 0.8181 - 2s/epoch - 9ms/step
Epoch 3/150
190/190 - 2s - loss: 0.2694 - accuracy: 0.8940 - val_loss: 0.3930 - val_accuracy: 0.8346 - 2s/epoch - 10ms/step
Epoch 4/150
190/190 - 2s - loss: 0.2027 - accuracy: 0.9230 - val_loss: 0.4151 - val_accuracy: 0.8457 - 2s/epoch - 9ms/step
Epoch 5/150
190/190 - 2s - loss: 0.1676 - accuracy: 0.9373 - val_loss: 0.4253 - val_accuracy: 0.8465 - 2s/epoch - 10ms/step
Epoch 6/150
190/190 - 2s - loss: 0.1325 - accuracy: 0.9531 - val_loss: 0.4830 - val_accuracy: 0.8480 - 2s/epoch - 9ms/step
Epoch 7/150
190/190 - 2s - loss: 0.1245 - accuracy: 0.9571 - val_loss: 0.5045 - val_accuracy: 0.8342 - 2s/epoch - 9ms/step
Epoch 8/150
190/190 - 2s - loss: 0.1145 - accuracy: 0.9597 - val_loss: 0.4846 - val_accuracy: 0.8438 - 2s/epoch - 9ms/step
Epoch 9/150
1

190/190 - 2s - loss: 0.0335 - accuracy: 0.9832 - val_loss: 1.0676 - val_accuracy: 0.8534 - 2s/epoch - 9ms/step
Epoch 68/150
190/190 - 2s - loss: 0.0324 - accuracy: 0.9829 - val_loss: 1.1006 - val_accuracy: 0.8511 - 2s/epoch - 9ms/step
Epoch 69/150
190/190 - 2s - loss: 0.0379 - accuracy: 0.9819 - val_loss: 1.0220 - val_accuracy: 0.8496 - 2s/epoch - 9ms/step
Epoch 70/150
190/190 - 2s - loss: 0.0363 - accuracy: 0.9819 - val_loss: 1.0669 - val_accuracy: 0.8438 - 2s/epoch - 9ms/step
Epoch 71/150
190/190 - 2s - loss: 0.0361 - accuracy: 0.9831 - val_loss: 1.0479 - val_accuracy: 0.8553 - 2s/epoch - 8ms/step
Epoch 72/150
190/190 - 2s - loss: 0.0347 - accuracy: 0.9831 - val_loss: 1.0663 - val_accuracy: 0.8484 - 2s/epoch - 9ms/step
Epoch 73/150
190/190 - 2s - loss: 0.0345 - accuracy: 0.9834 - val_loss: 1.0605 - val_accuracy: 0.8584 - 2s/epoch - 9ms/step
Epoch 74/150
190/190 - 2s - loss: 0.0326 - accuracy: 0.9840 - val_loss: 1.0997 - val_accuracy: 0.8496 - 2s/epoch - 9ms/step
Epoch 75/150
190/190 

Epoch 133/150
190/190 - 2s - loss: 0.0273 - accuracy: 0.9850 - val_loss: 1.4799 - val_accuracy: 0.8354 - 2s/epoch - 9ms/step
Epoch 134/150
190/190 - 2s - loss: 0.0325 - accuracy: 0.9840 - val_loss: 1.2275 - val_accuracy: 0.8530 - 2s/epoch - 9ms/step
Epoch 135/150
190/190 - 2s - loss: 0.0313 - accuracy: 0.9849 - val_loss: 1.2265 - val_accuracy: 0.8469 - 2s/epoch - 9ms/step
Epoch 136/150
190/190 - 2s - loss: 0.0269 - accuracy: 0.9855 - val_loss: 1.3256 - val_accuracy: 0.8515 - 2s/epoch - 9ms/step
Epoch 137/150
190/190 - 2s - loss: 0.0280 - accuracy: 0.9855 - val_loss: 1.4076 - val_accuracy: 0.8365 - 2s/epoch - 9ms/step
Epoch 138/150
190/190 - 2s - loss: 0.0296 - accuracy: 0.9852 - val_loss: 1.2983 - val_accuracy: 0.8542 - 2s/epoch - 9ms/step
Epoch 139/150
190/190 - 2s - loss: 0.0273 - accuracy: 0.9857 - val_loss: 1.3852 - val_accuracy: 0.8419 - 2s/epoch - 9ms/step
Epoch 140/150
190/190 - 2s - loss: 0.0283 - accuracy: 0.9850 - val_loss: 1.3852 - val_accuracy: 0.8411 - 2s/epoch - 9ms/step


In [163]:
yhat = model.predict(valid_padded1)
yhat = np.round(yhat).reshape(-1)

print(classification_report(y1_test, yhat))

              precision    recall  f1-score   support

           0       0.86      0.82      0.84      1305
           1       0.83      0.87      0.85      1301

    accuracy                           0.84      2606
   macro avg       0.84      0.84      0.84      2606
weighted avg       0.84      0.84      0.84      2606



In [164]:
model.save('NLP_tweets.h5')
print('Saved!!!')

Saved!!!


# Under Resample dataset

In [166]:
data['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [169]:
y_train.value_counts()

0    3468
1    2622
Name: target, dtype: int64

In [171]:
from imblearn.under_sampling import RandomUnderSampler

rs = RandomUnderSampler()
X_train_rs, y_train_rs = rs.fit_resample(X_train_cv, y_train)

In [172]:
y_train_rs.value_counts()

1    2622
0    2622
Name: target, dtype: int64

In [175]:
multiNB = MultinomialNB()
multiNB.fit(X_train_rs, y_train_rs)

print('Train score : {}'.format(multiNB.score(X_train_rs, y_train_rs)))
print('Test score : {}'.format(multiNB.score(X_test_cv, y_test)))

Train score : 0.9324942791762014
Test score : 0.7800393959290873


In [176]:
multiNB_score = cross_val_score(multiNB, X_train_rs, y_train_rs, cv=5)
multiNB_score.mean()

0.7837518829273973

In [177]:
fc.eval_clf_testset(multiNB, X_test_cv, y_test)

----- CLASSIFICATION MODEL PREFOMANCE IN TEST SET-----
* R-squared model of Test: 0.78

* Confusion Matrix of Test: 
[[685 189]
 [146 503]]

* Classification Report of Test: 
              precision    recall  f1-score   support

           0       0.82      0.78      0.80       874
           1       0.73      0.78      0.75       649

    accuracy                           0.78      1523
   macro avg       0.78      0.78      0.78      1523
weighted avg       0.78      0.78      0.78      1523



In [178]:
ber_nb = BernoulliNB()
ber_nb.fit(X_train_rs, y_train_rs)

print('Train score : {}'.format(ber_nb.score(X_train_rs, y_train_rs)))
print('Test score : {}'.format(ber_nb.score(X_test_cv, y_test)))

Train score : 0.9200991609458429
Test score : 0.7872619829284307


In [179]:
logit = LogisticRegression(random_state = 42)
logit.fit(X_train_rs, y_train_rs)

print('Train score : {}'.format(logit.score(X_train_rs, y_train_rs)))
print('Test score : {}'.format(logit.score(X_test_cv, y_test)))

Train score : 0.9759725400457666
Test score : 0.783322390019698


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])